# Exp hybrid 2

In [1]:
!pip install kaggle wandb onnx -Uq
from google.colab import drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 85.2 MB/s eta 0:00:00
Mounted at /content/drive


In [2]:
! mkdir ~/.kaggle
!cp /content/drive/MyDrive/ColabNotebooks/kaggle_API_credentials/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c walmart-recruiting-store-sales-forecasting
! unzip walmart-recruiting-store-sales-forecasting.zip
!unzip train.csv.zip
!unzip features.csv.zip

  0% 0.00/2.70M [00:00<?, ?B/s]
100% 2.70M/2.70M [00:00<00:00, 759MB/s]
Archive:  walmart-recruiting-store-sales-forecasting.zip
  inflating: features.csv.zip        
  inflating: sampleSubmission.csv.zip  
  inflating: stores.csv              
  inflating: test.csv.zip            
  inflating: train.csv.zip           
Archive:  train.csv.zip
  inflating: train.csv               
Archive:  features.csv.zip
  inflating: features.csv            


In [28]:
!unzip test.csv.zip

Archive:  test.csv.zip
  inflating: test.csv                


In [3]:
!pip install torch mlflow dagshub scikit-learn pandas numpy matplotlib seaborn joblib -q wandb torch torchvision torchaudio -q prophet neuralforecast

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261

In [4]:
#!/usr/bin/env python3
"""
Walmart Sales Forecasting - Hybrid Model (TFT + Prophet)
Adaptive weighting:
- Holiday periods: 60% Prophet + 40% TFT
- Regular days: 65% TFT + 35% Prophet
"""

import warnings
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error
from neuralforecast import NeuralForecast
from neuralforecast.models import TFT
from prophet import Prophet
from statsmodels.tools.sm_exceptions import ValueWarning
import zipfile
import os
import logging
from datetime import datetime, timedelta
import mlflow
import mlflow.sklearn

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore", category=ValueWarning)
warnings.filterwarnings("ignore")
logging.getLogger('prophet').setLevel(logging.WARNING)
logging.getLogger('cmdstanpy').setLevel(logging.WARNING)
logging.getLogger('cmdstanpy').setLevel(logging.ERROR)
# Additional cmdstanpy suppression
import logging
logging.getLogger('cmdstanpy').disabled = True
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)

# Data path configuration
KAGGLE_DATA_PATH = "/kaggle/input/walmart-recruiting-store-sales-forecasting/"

def calculate_wmae(y_true, y_pred, is_holiday_flag, holiday_weight=5.0):
    """Calculate Weighted Mean Absolute Error (WMAE) as per competition rules."""
    abs_errors = np.abs(y_true - y_pred)
    weights = np.where(is_holiday_flag.astype(bool), holiday_weight, 1.0)
    wmae = np.sum(weights * abs_errors) / np.sum(weights)
    return wmae

# =============================================================================
# TFT Components (from model-exp-tft-fx.ipynb)
# =============================================================================

class DateFeatureCreator(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        if "Date" not in X.columns:
            raise ValueError("DateFeatureCreator requires 'Date' column in input X.")

        # Ensure 'Date' is datetime type before operations
        if not pd.api.types.is_datetime64_any_dtype(X['Date']):
            X['Date'] = pd.to_datetime(X['Date'])

        # Using to_period('W') and then converting to integer week number
        # rank(method="dense") ensures consecutive integers for weeks
        X["week"] = (X["Date"].dt.to_period("W").rank(method="dense").astype(int) - 1)

        # Cyclical features for different periodicities
        X["sin_13"] = np.sin(2 * np.pi * X["week"] / 13) # Roughly quarterly seasonality
        X["cos_13"] = np.cos(2 * np.pi * X["week"] / 13)
        X["sin_23"] = np.sin(2 * np.pi * X["week"] / 23) # A different, less common periodicity
        X["cos_23"] = np.cos(2 * np.pi * X["week"] / 23)

        # Drop the original 'Date' column as its information is now in cyclical features
        X = X.drop(columns=["Date"], errors='ignore')
        return X

class ColumnDropper(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X.drop(columns=self.columns, errors="ignore")

class ColumnTransformerWithNames(ColumnTransformer):
    """
    A wrapper around ColumnTransformer to retain column names and return a DataFrame.
    Handles OneHotEncoder output specifically.
    """
    def __init__(self, transformers, remainder='drop'):
        super().__init__(transformers=transformers, remainder=remainder)
        self.output_columns_ = None

    def fit(self, X, y=None):
        super().fit(X, y)
        self.output_columns_ = self._get_feature_names_out_internal(X)
        return self

    def _get_feature_names_out_internal(self, X):
        column_names = []
        for name, transformer, columns in self.transformers_:
            if transformer == 'drop':
                continue
            elif transformer == 'passthrough':
                # Ensure passthrough columns are correctly identified from original X
                if isinstance(columns, str):
                    column_names.append(columns)
                else:
                    column_names.extend(list(columns))
            else:
                if hasattr(transformer, 'get_feature_names_out'):
                    if isinstance(columns, str):
                        col_names = [columns]
                    else:
                        col_names = list(columns)
                    column_names.extend(list(transformer.get_feature_names_out(col_names)))
                else:
                    if isinstance(columns, str): # Fallback for transformers without get_feature_names_out
                        column_names.append(columns)
                    else:
                        column_names.extend(list(columns))
        return column_names

    def transform(self, X):
        transformed_array = super().transform(X)
        if self.output_columns_ is None:
             raise RuntimeError("ColumnTransformerWithNames must be fitted before transform.")

        # Convert to dense array if it's a sparse matrix (older sklearn default for OHE)
        if hasattr(transformed_array, 'toarray'):
            transformed_array = transformed_array.toarray()

        # Ensure that the index is preserved from the input X
        # This is CRITICAL for maintaining alignment with y
        return pd.DataFrame(transformed_array, index=X.index, columns=self.output_columns_)

    def fit_transform(self, X, y=None):
        transformed_array = super().fit_transform(X, y)
        self.output_columns_ = self._get_feature_names_out_internal(X)

        # Convert to dense array if it's a sparse matrix (older sklearn default for OHE)
        if hasattr(transformed_array, 'toarray'):
            transformed_array = transformed_array.toarray()

        # Ensure that the index is preserved from the input X
        # This is CRITICAL for maintaining alignment with y
        return pd.DataFrame(transformed_array, index=X.index, columns=self.output_columns_)

class MultiIndexKeeper(BaseEstimator, TransformerMixin):
    def __init__(self, index_cols=["Date", "Store", "Dept"]):
        self.index_cols = index_cols

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()

        if 'Date' in X.columns and not pd.api.types.is_datetime64_any_dtype(X['Date']):
            X['Date'] = pd.to_datetime(X['Date'])

        missing_cols = [col for col in self.index_cols if col not in X.columns]
        if missing_cols:
            raise ValueError(f"MultiIndexKeeper: Missing columns in input X: {missing_cols}")

        # IMPORTANT: When setting index, ensure 'Date', 'Store', 'Dept' columns
        # are not dropped, as they are needed later by NeuralForecast as covariates.
        # This is already handled by drop=False.
        X.set_index(self.index_cols, drop=False, inplace=True)
        return X

class TFTRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, input_chunk_length=52, output_chunk_length=39, epochs=25, batch_size=32, random_seed=42):
        self.input_chunk_length = input_chunk_length
        self.output_chunk_length = output_chunk_length
        self.epochs = epochs
        self.batch_size = batch_size
        self.random_seed = random_seed
        self.nf_ = None
        self.model_ = None
        self.trained_df_ = None # Store the DataFrame used for training

    def fit(self, X, y):
        # Ensure y has a name for proper merging if it doesn't already
        if y.name is None:
            y.name = 'y'

        y_multiindexed = pd.Series(y.values, index=X.index, name='y')

        df = X.copy()
        df['y'] = y_multiindexed

        if not pd.api.types.is_datetime64_any_dtype(df.index.get_level_values('Date')):
            raise ValueError("MultiIndex 'Date' level is not datetime type. Ensure MultiIndexKeeper makes it datetime.")
        df['ds'] = df.index.get_level_values('Date')

        df["unique_id"] = df["Store"].astype(str) + "_" + df["Dept"].astype(str)

        # Store the prepared DataFrame for use in predict
        self.trained_df_ = df.copy() # Store the full df (including y and features)

        # --- DEBUGGING STEP: Check for NaNs immediately before NeuralForecast fit ---
        nan_check = df.isnull().sum()
        cols_with_nans = nan_check[nan_check > 0].index.tolist()
        if cols_with_nans:
            print(f"DEBUG: Found NaNs in the following columns before NeuralForecast fit: {cols_with_nans}")
            # Print head including index for better context
            print(df.loc[df[cols_with_nans[0]].isnull(), cols_with_nans].head())
            raise ValueError(f"Found missing values in {cols_with_nans}.")
        # --- END DEBUGGING STEP ---

        self.model_ = TFT(
            h=self.output_chunk_length,
            input_size=self.input_chunk_length,
            batch_size=self.batch_size,
            random_seed=self.random_seed,
        )

        self.nf_ = NeuralForecast(models=[self.model_], freq="W-FRI")

        self.nf_.fit(df=df)
        return self

    def predict(self, X):
        # X here is the transformed X_val from the pipeline, with MultiIndex ('Date', 'Store', 'Dept')

        # 1. Prepare future covariates from X (which is X_val after preprocessing)
        df_future_covariates_raw = X.copy() # This X contains all features for the validation period

        if not pd.api.types.is_datetime64_any_dtype(df_future_covariates_raw.index.get_level_values('Date')):
            raise ValueError("MultiIndex 'Date' level is not datetime type in predict. Ensure MultiIndexKeeper makes it datetime.")

        df_future_covariates_raw['ds'] = df_future_covariates_raw.index.get_level_values('Date')
        df_future_covariates_raw["unique_id"] = df_future_covariates_raw["Store"].astype(str) + "_" + df_future_covariates_raw["Dept"].astype(str)

        # Identify all covariate columns (all columns in self.trained_df_ except 'ds', 'unique_id', 'y')
        covariate_cols = [col for col in self.trained_df_.columns if col not in ['ds', 'unique_id', 'y']]

        # Select only the relevant future covariate columns and the required 'ds', 'unique_id'
        df_future_covariates_selected = df_future_covariates_raw[['ds', 'unique_id'] + covariate_cols].copy()

        # 2. Generate the full expected future dataframe for all series for the forecast horizon
        expected_future_df_template = self.nf_.make_future_dataframe(self.trained_df_)

        # 3. Merge the generated template with our actual future covariates (X_val)
        futr_df_complete = pd.merge(
            expected_future_df_template,
            df_future_covariates_selected,
            on=['unique_id', 'ds'],
            how='left'
        )

        nan_check_futr = futr_df_complete.isnull().sum()
        cols_with_nans_futr = nan_check_futr[nan_check_futr > 0].index.tolist()
        if cols_with_nans_futr:
            print(f"DEBUG: Found NaNs in futr_df_complete for columns: {cols_with_nans_futr}. Filling with 0.")
            futr_df_complete[cols_with_nans_futr] = futr_df_complete[cols_with_nans_futr].fillna(0)

        # 4. Perform the prediction
        # Ensure that `df` has all required unique_ids and `futr_df` aligns.
        # This is the point where the model generates predictions.
        forecast_df = self.nf_.predict(df=self.trained_df_, futr_df=futr_df_complete)

        forecast_df = forecast_df.rename(columns={'TFT': 'yhat'})

        if 'unique_id' in forecast_df.columns:
            forecast_df[['Store', 'Dept']] = forecast_df['unique_id'].str.split('_', expand=True)
            forecast_df['Store'] = forecast_df['Store'].astype(float).astype(int)
            forecast_df['Dept'] = forecast_df['Dept'].astype(float).astype(int)

        forecast_df['Date'] = pd.to_datetime(forecast_df['ds'])

        # Ensure the index columns are correct before setting the index
        # Also, make sure that 'Store' and 'Dept' are properly integer type before setting multi-index
        forecast_df_indexed = forecast_df.set_index(['Date', 'Store', 'Dept'])[['yhat']]

        # This is where NaNs can be introduced if forecast_df_indexed doesn't cover all X.index
        final_predictions = forecast_df_indexed.reindex(X.index)

        y_pred = final_predictions['yhat'].values.flatten()

        # FIX: Fill any NaNs in the final predictions array with 0 before evaluation
        if np.isnan(y_pred).any():
            print("DEBUG: Found NaNs in final y_pred after reindex. Filling with 0.")
            y_pred = np.nan_to_num(y_pred, nan=0.0)

        y_pred[y_pred < 0] = 0

        return y_pred

# =============================================================================
# Prophet Components (from model_exp_FX_Prophet.ipynb)
# =============================================================================

class WalmartProphetPreprocessingPipeline:
    """
    Preprocessing pipeline for Prophet models.
    Focuses on preparing data in the 'ds' (Date) and 'y' (Weekly_Sales) format,
    and handling holidays.
    """

    def __init__(self):
        self.fitted = False
        self.holidays_df = None

    def load_and_prepare_data(self):
        """Load and merge necessary datasets for Prophet."""
        print("📊 Loading datasets...")

        # Load datasets
        train_df = pd.read_csv('train.csv')
        features_df = pd.read_csv('features.csv')
        stores_df = pd.read_csv('stores.csv')

        print(f"   📈 Train data: {train_df.shape}")
        print(f"   📊 Features data: {features_df.shape}")
        print(f"   🏪 Stores data: {stores_df.shape}")

        # Convert Date columns to datetime
        train_df['Date'] = pd.to_datetime(train_df['Date'])
        features_df['Date'] = pd.to_datetime(features_df['Date'])

        # Merge datasets (similar to previous steps)
        merged_df = pd.merge(train_df, features_df, on=['Store', 'Date', 'IsHoliday'], how='left')
        train_full = pd.merge(merged_df, stores_df, on=['Store'], how='left')

        # Sort by date for time series consistency
        train_full = train_full.sort_values(by=['Store', 'Dept', 'Date']).reset_index(drop=True)

        print(f"   ✅ Merged data: {train_full.shape}")
        print(f"   📅 Date range: {train_full['Date'].min()} to {train_full['Date'].max()}")

        return train_full

    def create_temporal_split(self, df, train_ratio=0.8):
        """Create temporal split to prevent data leakage for time series."""
        print(f"📅 Creating temporal split ({int(train_ratio*100)}/{int((1-train_ratio)*100)})...")

        # Sort by date to ensure temporal order
        df_sorted = df.sort_values('Date').reset_index(drop=True)

        # Determine split date based on a fixed ratio of unique dates
        unique_dates = sorted(df_sorted['Date'].unique())
        total_weeks = len(unique_dates)
        train_weeks = int(total_weeks * train_ratio)

        # Ensure train_weeks is at least 1, and not exceeding total_weeks
        if train_weeks < 1:
            train_weeks = 1
        if train_weeks >= total_weeks:
            train_weeks = total_weeks - 1 # Ensure at least one week for validation/test

        split_date = unique_dates[train_weeks - 1] # End date of training

        # Split data
        train_data = df_sorted[df_sorted['Date'] <= split_date].copy()
        val_data = df_sorted[df_sorted['Date'] > split_date].copy()

        # Create split info dictionary
        split_info = {
            'split_date': split_date,
            'train_size': len(train_data),
            'val_size': len(val_data),
            'train_date_range': (train_data['Date'].min(), train_data['Date'].max()),
            'val_date_range': (val_data['Date'].min(), val_data['Date'].max())
        }

        print(f"   📊 Split date: {split_date}")
        print(f"   📈 Train: {len(train_data):,} records ({train_data['Date'].min()} to {train_data['Date'].max()})")
        print(f"   📉 Val: {len(val_data):,} records ({val_data['Date'].min()} to {val_data['Date'].max()})")

        return train_data, val_data, split_info

    def fit(self, train_data):
        """Fit the preprocessing pipeline (prepare holidays)."""
        print("🔧 Preparing Prophet specific data (holidays)...")

        # Prophet's holidays DataFrame: requires 'holiday', 'ds' columns
        # We define common US holidays that align with Walmart's IsHoliday flag
        self.holidays_df = pd.DataFrame([
            # Super Bowl: IsHoliday=True
            {'holiday': 'SuperBowl', 'ds': '2010-02-12'},
            {'holiday': 'SuperBowl', 'ds': '2011-02-11'},
            {'holiday': 'SuperBowl', 'ds': '2012-02-10'},

            # Labor Day: IsHoliday=True
            {'holiday': 'LaborDay', 'ds': '2010-09-10'},
            {'holiday': 'LaborDay', 'ds': '2011-09-09'},
            {'holiday': 'LaborDay', 'ds': '2012-09-07'},

            # Thanksgiving: IsHoliday=True
            {'holiday': 'Thanksgiving', 'ds': '2010-11-26'},
            {'holiday': 'Thanksgiving', 'ds': '2011-11-25'},
            {'holiday': 'Thanksgiving', 'ds': '2012-11-23'},

            # Christmas: IsHoliday=True
            {'holiday': 'Christmas', 'ds': '2010-12-31'},
            {'holiday': 'Christmas', 'ds': '2011-12-30'},
            {'holiday': 'Christmas', 'ds': '2012-12-28'}
        ])

        # Convert ds to datetime
        self.holidays_df['ds'] = pd.to_datetime(self.holidays_df['ds'])

        self.fitted = True
        print("✅ Pipeline fitted on training data with holiday-aware settings")

        return self

    def transform(self, data, is_validation=False):
        """Transform data for Prophet format."""
        if not self.fitted:
            raise ValueError("Pipeline must be fitted before transforming data.")

        print("🔄 Transforming training data..." if not is_validation else "🔄 Transforming validation data...")

        # Prophet requires very specific column names: 'ds', 'y'
        prophet_data = data.copy()

        # Ensure we have all necessary columns
        required_cols = ['Date', 'Weekly_Sales', 'Store', 'Dept', 'IsHoliday']
        if not all(col in prophet_data.columns for col in required_cols):
            missing = [col for col in required_cols if col not in prophet_data.columns]
            raise ValueError(f"Missing required columns: {missing}")

        # Rename columns for Prophet
        prophet_data = prophet_data.rename(columns={
            'Date': 'ds',
            'Weekly_Sales': 'y'
        })

        # Ensure ds is datetime
        prophet_data['ds'] = pd.to_datetime(prophet_data['ds'])

        # Keep only necessary columns for Prophet
        prophet_data = prophet_data[['ds', 'y', 'Store', 'Dept', 'IsHoliday']].copy()

        print(f"✅ Transform complete. Shape: {prophet_data.shape}")

        return prophet_data

    def fit_transform(self, train_data):
        """Convenience method to fit and transform in one step."""
        return self.fit(train_data).transform(train_data, is_validation=False)

    def get_preprocessed_data(self):
        """
        Orchestrates preprocessing steps to get model-ready data.

        Returns:
            train_data_prophet, val_data_prophet: DataFrames ready for Prophet
            split_info: Information about the temporal split
            holidays_df: DataFrame of holidays for Prophet
        """
        print("🔄 Getting preprocessed data using pipeline...")

        # Create the preprocessing pipeline
        pipeline = WalmartProphetPreprocessingPipeline()

        # Load raw data
        train_full = pipeline.load_and_prepare_data()

        # Create temporal split
        train_data_raw, val_data_raw, split_info = pipeline.create_temporal_split(train_full)

        # Fit and transform data using pipeline
        pipeline.fit(train_data_raw)
        train_data_prophet = pipeline.transform(train_data_raw, is_validation=False)
        val_data_prophet = pipeline.transform(val_data_raw, is_validation=True)

        holidays_df = pipeline.holidays_df # Get holidays after fitting pipeline

        print(f"✅ Data preprocessing complete!")
        print(f"   📊 Training shape: {train_data_prophet.shape}")
        print(f"   📊 Validation shape: {val_data_prophet.shape}")

        return train_data_prophet, val_data_prophet, split_info, holidays_df

def train_prophet_models(train_data_prophet, holidays_df, min_observations=50):
    """
    Train Prophet models for each Store-Dept combination.

    Args:
        train_data_prophet: Training data in Prophet format (ds, y, Store, Dept, IsHoliday)
        holidays_df: DataFrame of holidays for Prophet
        min_observations: Minimum number of observations required to train a model

    Returns:
        dict: Dictionary with (Store, Dept) as keys and trained Prophet models as values
    """
    print("📈 Training Prophet models for each Store-Dept combination...")

    # Get unique Store-Dept combinations
    unique_combinations = train_data_prophet[['Store', 'Dept']].drop_duplicates()
    total_combinations = len(unique_combinations)

    print(f"   ⏰ No time limit - training all combinations")
    print(f"   📊 Training models for {total_combinations} combinations")
    print(f"   🎯 Training Prophet for all combinations")

    models = {}
    successful_count = 0
    failed_count = 0

    for idx, (_, row) in enumerate(unique_combinations.iterrows(), 1):
        store_id = row['Store']
        dept_id = row['Dept']

        # Filter data for this specific Store-Dept combination
        series_data = train_data_prophet[
            (train_data_prophet['Store'] == store_id) &
            (train_data_prophet['Dept'] == dept_id)
        ].copy()

        # Check if we have enough data points
        if len(series_data) < min_observations:
            failed_count += 1
            continue

        try:
            # Initialize Prophet model with holidays
            m = Prophet(
                yearly_seasonality=True,
                weekly_seasonality=True,
                holidays=holidays_df
            )

            # Train the model on this series
            # Prophet expects only 'ds' and 'y' columns for fitting
            fit_data = series_data[['ds', 'y']].copy()
            m.fit(fit_data)

            # Store the trained model
            models[(store_id, dept_id)] = m
            successful_count += 1

        except Exception as e:
            # If training fails for this series, skip it
            failed_count += 1
            # Uncomment for debugging: print(f"   ❌ Failed to train model for Store {store_id}, Dept {dept_id}: {str(e)}")
            continue

        # Progress update every 200 models
        if idx % 200 == 0 or idx == 1:
            print(f"   ✅ Trained {idx}/{total_combinations} models ({successful_count} successful, {failed_count} failed)")

    # Final progress update
    print(f"   ✅ Trained {total_combinations}/{total_combinations} models ({successful_count} successful, {failed_count} failed)")

    print("✅ Prophet training complete!")
    print(f"   🎯 Successful models: {successful_count}")
    print(f"   ❌ Failed models: {failed_count}")
    print(f"   📊 Coverage: {successful_count}/{total_combinations} ({successful_count/total_combinations*100:.1f}%)")

    return models

def make_prophet_predictions(models, val_data_prophet, train_data=None):
    """
    Make predictions using trained Prophet models.

    Args:
        models: Dictionary of trained Prophet models
        val_data_prophet: Validation data in Prophet format
        train_data: Not used, kept for compatibility

    Returns:
        tuple: (predictions, actuals, holidays_flags) as numpy arrays
    """
    print("📈 Making Prophet predictions...")

    predictions = []
    actuals = []
    holidays_flags = []

    successful_predictions_count = 0
    skipped_predictions_no_model = 0
    failed_predictions_count = 0

    # Debug: Count total validation points
    total_val_points = len(val_data_prophet)
    print(f"   📊 Total validation points: {total_val_points}")
    print(f"   📊 Available models: {len(models)}")

    # Get unique Store-Dept combinations from validation data
    unique_combinations = val_data_prophet[['Store', 'Dept']].drop_duplicates()
    print(f"   📊 Unique Store-Dept combinations in validation: {len(unique_combinations)}")

    combinations_with_models = 0
    combinations_without_models = 0

    for _, row in unique_combinations.iterrows():
        store_id = row['Store']
        dept_id = row['Dept']

        # Filter validation data for this Store-Dept combination
        series_data = val_data_prophet[
            (val_data_prophet['Store'] == store_id) &
            (val_data_prophet['Dept'] == dept_id)
        ].copy()

        if series_data.empty:
            continue

        # Check if we have a trained model for this combination
        if (store_id, dept_id) not in models:
            # No model available - use zeros as predictions
            predictions.extend(np.zeros(len(series_data)))
            actuals.extend(series_data['y'].values)
            holidays_flags.extend(series_data['IsHoliday'].values)
            skipped_predictions_no_model += len(series_data)
            combinations_without_models += 1
            continue

        combinations_with_models += 1

        try:
            # Get the trained model
            m = models[(store_id, dept_id)]

            # Make predictions for this series
            # Prophet needs only 'ds' column for prediction
            forecast = m.predict(series_data[['ds']])

            # Extract predictions and ensure no negative values
            yhat = forecast['yhat'].values
            yhat[yhat < 0] = 0  # Ensure no negative predictions

            # Store results
            predictions.extend(yhat)
            actuals.extend(series_data['y'].values)
            holidays_flags.extend(series_data['IsHoliday'].values)
            successful_predictions_count += len(series_data)

        except Exception as e:
            # If prediction fails, use zeros
            predictions.extend(np.zeros(len(series_data)))
            actuals.extend(series_data['y'].values)
            holidays_flags.extend(series_data['IsHoliday'].values)
            failed_predictions_count += len(series_data)
            # Uncomment for debugging: print(f"   ❌ Failed to predict for Store {store_id}, Dept {dept_id}: {str(e)}")

    print("✅ Predictions complete!")
    print(f"   🎯 Prophet predictions: {successful_predictions_count}")
    print(f"   ⏭️ Skipped (no model): {skipped_predictions_no_model}")
    print(f"   ❌ Failed predictions: {failed_predictions_count}")
    print(f"   📊 Combinations with models: {combinations_with_models}")
    print(f"   📊 Combinations without models: {combinations_without_models}")
    print(f"   📊 Total predictions generated: {len(predictions)}")

    return np.array(predictions), np.array(actuals), np.array(holidays_flags).astype(bool)

def run_standalone_prophet_experiment():
    """
    Run the exact FX Prophet experiment standalone to get Prophet predictions.
    This replicates the exact FX Prophet notebook flow.

    Returns:
        tuple: (y_pred_val, y_true_val, is_holiday_val, prophet_wmae)
    """
    print("🚀 Running Standalone Prophet Experiment (FX Prophet Exact Copy)")
    print("=" * 80)

    try:
        # Step 1: Get preprocessed data (exact FX Prophet method)
        print("\n📊 Step 1: Data preprocessing...")
        train_data_prophet, val_data_prophet, split_info, holidays_df = WalmartProphetPreprocessingPipeline().get_preprocessed_data()

        # Step 2: Train Prophet models (exact FX Prophet method)
        print("\n📈 Step 2: Training Prophet models...")
        models = train_prophet_models(train_data_prophet, holidays_df)

        # Step 3: Make predictions on validation set (exact FX Prophet method)
        print("\n📈 Step 3: Making predictions...")
        y_pred_val, y_true_val, is_holiday_val = make_prophet_predictions(models, val_data_prophet)

        # Step 4: Calculate validation metrics (exact FX Prophet method)
        print("\n📊 Step 4: Calculating validation metrics...")

        # Validation metrics
        if len(y_true_val) > 0:
            val_mae = mean_absolute_error(y_true_val, y_pred_val)
            val_rmse = np.sqrt(mean_squared_error(y_true_val, y_pred_val))
            val_wmae = calculate_wmae(y_true_val, y_pred_val, is_holiday_val)
        else:
            val_mae, val_rmse, val_wmae = 0, 0, 0
            print("   ⚠️ Warning: No data points for evaluation. Metrics set to 0.")

        # Holiday breakdown for validation
        holiday_mask_val = is_holiday_val.astype(bool)
        holiday_mae_val = mean_absolute_error(y_true_val[holiday_mask_val], y_pred_val[holiday_mask_val]) if holiday_mask_val.any() else 0
        non_holiday_mae_val = mean_absolute_error(y_true_val[~holiday_mask_val], y_pred_val[~holiday_mask_val]) if (~holiday_mask_val).any() else 0

        # Print results (exact FX Prophet format)
        print("\n" + "=" * 60)
        print("🎯 STANDALONE PROPHET RESULTS (FX Prophet Copy)")
        print("=" * 60)

        print("📊 Validation Metrics:")
        print(f"   WMAE (Competition Metric): ${val_wmae:,.2f}")
        print(f"   MAE: ${val_mae:,.2f}")
        print(f"   RMSE: ${val_rmse:,.2f}")

        print("\n📊 Holiday Breakdown:")
        print(f"   Holiday MAE: ${holiday_mae_val:,.2f} ({np.sum(holiday_mask_val)} samples)")
        print(f"   Non-Holiday MAE: ${non_holiday_mae_val:,.2f} ({np.sum(~holiday_mask_val)} samples)")

        print("\n📊 Model Statistics:")
        print(f"   Successful models trained: {len(models):,}")
        print(f"   Predictions generated: {len(y_pred_val):,}")

        print("\n🎉 Standalone Prophet Experiment: COMPLETE!")

        return y_pred_val, y_true_val, is_holiday_val, val_wmae

    except Exception as e:
        print(f"❌ Standalone Prophet experiment failed: {e}")
        raise

# =============================================================================
# Hybrid Model Implementation
# =============================================================================

class HybridTFTProphetRegressor(BaseEstimator, RegressorMixin):
    """
    Hybrid model combining TFT and Prophet with adaptive weighting:
    - Holiday periods: 60% Prophet + 40% TFT
    - Regular days: 65% TFT + 35% Prophet
    """

    def __init__(self, input_chunk_length=52, output_chunk_length=39, epochs=25, batch_size=32,
                 random_seed=42, min_observations=50):
        # TFT parameters
        self.input_chunk_length = input_chunk_length
        self.output_chunk_length = output_chunk_length
        self.epochs = epochs
        self.batch_size = batch_size
        self.random_seed = random_seed

        # Prophet parameters
        self.min_observations = min_observations

        # Model components
        self.tft_model_ = None
        self.prophet_models_ = None
        self.prophet_predictions_ = None  # Store Prophet predictions from training
        self.prophet_actuals_ = None      # Store Prophet actuals
        self.prophet_holiday_flags_ = None # Store Prophet holiday flags
        self.prophet_wmae_ = None         # Store Prophet standalone WMAE
        self.tft_predictions_ = None      # Store TFT predictions

        # Hybrid weights
        self.holiday_prophet_weight = 0.60
        self.holiday_tft_weight = 0.40
        self.regular_tft_weight = 0.65
        self.regular_prophet_weight = 0.35

    def fit(self, X, y):
        """Fit Prophet first, then train TFT on the exact same data Prophet uses."""
        print("🤖 Training Hybrid Model (TFT + Prophet)...")

        # STEP 1: Run standalone Prophet experiment (exact FX Prophet copy)
        print("\n📊 Step 1: Running standalone Prophet experiment...")

        prophet_predictions, prophet_actuals, prophet_holiday_flags, prophet_wmae = run_standalone_prophet_experiment()

        # Store Prophet results
        self.prophet_predictions_ = prophet_predictions
        self.prophet_actuals_ = prophet_actuals
        self.prophet_holiday_flags_ = prophet_holiday_flags
        self.prophet_wmae_ = prophet_wmae

        print(f"\n✅ Prophet standalone experiment complete!")
        print(f"   📈 Prophet WMAE: ${prophet_wmae:,.2f}")
        print(f"   📊 Prophet predictions: {len(prophet_predictions):,}")

        # STEP 2: Get Prophet's exact data and preprocess it for TFT
        print("\n📊 Step 2: Preparing TFT to use Prophet's exact data...")

        # Get the same data split Prophet used
        pipeline = WalmartProphetPreprocessingPipeline()
        train_full = pipeline.load_and_prepare_data()
        train_data_raw, val_data_raw, split_info = pipeline.create_temporal_split(train_full)

        print(f"   📊 Prophet's data split - Train: {len(train_data_raw)}, Val: {len(val_data_raw)}")

        # CRITICAL FIX: Preprocess Prophet's data through the main pipeline to get TFT-compatible format
        print("   🔄 Converting Prophet's data to TFT format through main pipeline...")

        # We need to process Prophet's raw data through the main pipeline transformations
        # This ensures TFT gets the same data points as Prophet, just in TFT format

        # Extract features and target from Prophet's training data
        X_train_prophet = train_data_raw.drop(['Weekly_Sales'], axis=1)
        y_train_prophet = train_data_raw['Weekly_Sales']

        # Store Prophet's validation data for TFT prediction
        X_val_prophet = val_data_raw.drop(['Weekly_Sales'], axis=1)
        y_val_prophet = val_data_raw['Weekly_Sales']

        print(f"   📊 TFT will use Prophet's exact data: Train {X_train_prophet.shape}, Val {X_val_prophet.shape}")

        # STEP 3: Train TFT on Prophet's exact data
        print("\n📊 Step 3: Training TFT on Prophet's exact data...")

        self.tft_model_ = TFTRegressor(
            input_chunk_length=self.input_chunk_length,
            output_chunk_length=self.output_chunk_length,
            epochs=self.epochs,
            batch_size=self.batch_size,
            random_seed=self.random_seed
        )

        # Train TFT on Prophet's exact training data
        self.tft_model_.fit(X_train_prophet, y_train_prophet)

        # Store Prophet's validation data for TFT prediction
        self.X_val_prophet_ = X_val_prophet
        self.y_val_prophet_ = y_val_prophet

        print("✅ TFT model training complete!")
        print("✅ Hybrid model training complete!")
        print("   🎯 Both models trained on identical data points!")

        return self

    def predict(self, X):
        """Make hybrid predictions using both models trained on Prophet's exact data."""
        print("🔮 Making hybrid predictions...")

        # Check if we have Prophet predictions from training
        if self.prophet_predictions_ is None:
            raise ValueError("Prophet predictions not available. Train the model first.")

        # STEP 1: Use pre-computed Prophet predictions from standalone experiment
        print("   📊 Using standalone Prophet predictions...")
        prophet_predictions = self.prophet_predictions_
        prophet_actuals = self.prophet_actuals_
        prophet_holiday_flags = self.prophet_holiday_flags_

        print(f"   📊 Prophet standalone WMAE: ${self.prophet_wmae_:,.2f}")
        print(f"   📊 Prophet predictions: {len(prophet_predictions):,}")

        # STEP 2: Get TFT predictions on Prophet's exact validation data
        print("   📊 Getting TFT predictions on Prophet's exact validation data...")

        # Use Prophet's exact validation data for TFT prediction
        if hasattr(self, 'X_val_prophet_'):
            tft_predictions = self.tft_model_.predict(self.X_val_prophet_)
            print(f"   📊 TFT predictions on Prophet's exact data: {len(tft_predictions):,}")
        else:
            # Fallback (shouldn't happen)
            print("   ⚠️ Warning: Prophet's validation data not available, using provided X")
            tft_predictions = self.tft_model_.predict(X)

        # STEP 3: Verify perfect alignment (should be the same length now)
        print(f"   ✅ Checking alignment - Prophet: {len(prophet_predictions)}, TFT: {len(tft_predictions)}")

        if len(prophet_predictions) != len(tft_predictions):
            print(f"   ⚠️ Unexpected length mismatch - Prophet: {len(prophet_predictions)}, TFT: {len(tft_predictions)}")
            # Truncate to minimum length
            min_len = min(len(prophet_predictions), len(tft_predictions))
            prophet_predictions = prophet_predictions[:min_len]
            tft_predictions = tft_predictions[:min_len]
            prophet_holiday_flags = prophet_holiday_flags[:min_len]
            prophet_actuals = prophet_actuals[:min_len]
            print(f"   ✂️ Truncated to length: {min_len}")
        else:
            print(f"   🎯 Perfect alignment achieved: {len(prophet_predictions)} predictions")

        # Store individual predictions for evaluation
        self.tft_predictions_ = tft_predictions.copy()
        self.prophet_predictions_aligned_ = prophet_predictions.copy()
        self.prophet_actuals_aligned_ = prophet_actuals.copy()
        self.prophet_holiday_flags_aligned_ = prophet_holiday_flags.copy()

        # STEP 4: Apply adaptive weighting using Prophet's holiday flags
        print("   🎯 Applying adaptive weighting...")

        is_holiday = prophet_holiday_flags.astype(bool)

        # Initialize hybrid predictions
        hybrid_predictions = np.zeros_like(tft_predictions, dtype=float)

        # Apply different weights for holidays vs regular days
        holiday_mask = is_holiday
        regular_mask = ~is_holiday

        # Holiday periods: 60% Prophet + 40% TFT
        if np.any(holiday_mask):
            hybrid_predictions[holiday_mask] = (
                self.holiday_prophet_weight * prophet_predictions[holiday_mask] +
                self.holiday_tft_weight * tft_predictions[holiday_mask]
            )

        # Regular days: 65% TFT + 35% Prophet
        if np.any(regular_mask):
            hybrid_predictions[regular_mask] = (
                self.regular_tft_weight * tft_predictions[regular_mask] +
                self.regular_prophet_weight * prophet_predictions[regular_mask]
            )

        # Ensure no negative predictions
        hybrid_predictions[hybrid_predictions < 0] = 0

        # Report weighting statistics
        n_holiday = np.sum(holiday_mask)
        n_regular = np.sum(regular_mask)
        total_points = len(hybrid_predictions)

        print(f"   📊 Weighting applied:")
        print(f"      Holiday points: {n_holiday:,} ({n_holiday/total_points*100:.1f}%) - 60% Prophet + 40% TFT")
        print(f"      Regular points: {n_regular:,} ({n_regular/total_points*100:.1f}%) - 65% TFT + 35% Prophet")

        print("✅ Hybrid predictions complete!")
        print("   🎯 Both models trained and predicted on identical data!")

        return hybrid_predictions

def evaluate_individual_models(hybrid_model, X_val, y_val, is_holiday_val):
    """
    Evaluate individual TFT and Prophet models within the hybrid model.

    Args:
        hybrid_model: Trained HybridTFTProphetRegressor
        X_val: Validation features (not used for Prophet)
        y_val: Validation targets (aligned) - Prophet's raw targets
        is_holiday_val: Holiday flags for validation (aligned)

    Returns:
        dict: Dictionary containing individual model metrics
    """
    print("\n📊 Individual Model Evaluation:")

    # Get stored predictions from hybrid model (these are already aligned)
    tft_pred = hybrid_model.tft_predictions_
    prophet_pred = hybrid_model.prophet_predictions_aligned_

    # Use aligned actuals and holiday flags
    if hasattr(hybrid_model, 'prophet_actuals_aligned_'):
        prophet_actuals = hybrid_model.prophet_actuals_aligned_
        prophet_holiday_flags = hybrid_model.prophet_holiday_flags_aligned_
    else:
        prophet_actuals = hybrid_model.prophet_actuals_
        prophet_holiday_flags = hybrid_model.prophet_holiday_flags_

    # CRITICAL FIX: TFT and Prophet should be evaluated against the same targets
    # Since we're using Prophet's validation period, use Prophet's actuals for both

    # Ensure all predictions match validation data length
    min_len = min(len(tft_pred), len(prophet_pred), len(prophet_actuals))
    tft_pred = tft_pred[:min_len]
    prophet_pred = prophet_pred[:min_len]
    actuals_for_evaluation = prophet_actuals[:min_len]  # Use Prophet's actuals for both
    is_holiday_trimmed = prophet_holiday_flags[:min_len] if len(prophet_holiday_flags) >= min_len else np.zeros(min_len, dtype=bool)

    print(f"   📊 Evaluating both models against Prophet's actuals (same scale)")
    print(f"   📊 Evaluation samples: {len(actuals_for_evaluation):,}")

    # DEBUG: Print prediction statistics to understand scale differences
    print(f"\n   🔍 DEBUG - Prediction Statistics:")
    print(f"      Actuals: min={np.min(actuals_for_evaluation):.2f}, max={np.max(actuals_for_evaluation):.2f}, mean={np.mean(actuals_for_evaluation):.2f}")
    print(f"      Prophet: min={np.min(prophet_pred):.2f}, max={np.max(prophet_pred):.2f}, mean={np.mean(prophet_pred):.2f}")
    print(f"      TFT: min={np.min(tft_pred):.2f}, max={np.max(tft_pred):.2f}, mean={np.mean(tft_pred):.2f}")

    # Check if TFT predictions are on a completely different scale
    tft_scale_factor = np.mean(actuals_for_evaluation) / np.mean(tft_pred) if np.mean(tft_pred) != 0 else 1
    print(f"      Scale factor (actuals/TFT): {tft_scale_factor:.4f}")

    if abs(tft_scale_factor - 1.0) > 0.5:  # If scale difference > 50%
        print(f"   ⚠️ WARNING: TFT predictions appear to be on a different scale!")
        print(f"   💡 This suggests TFT was trained on preprocessed targets while being evaluated on raw targets")

    # Calculate TFT metrics (against Prophet's actuals - same scale as Prophet)
    tft_wmae = calculate_wmae(actuals_for_evaluation, tft_pred, is_holiday_trimmed)
    tft_mae = mean_absolute_error(actuals_for_evaluation, tft_pred)
    tft_rmse = np.sqrt(mean_squared_error(actuals_for_evaluation, tft_pred))

    # Calculate Prophet metrics (against its own actuals)
    prophet_wmae = calculate_wmae(actuals_for_evaluation, prophet_pred, is_holiday_trimmed)
    prophet_mae = mean_absolute_error(actuals_for_evaluation, prophet_pred)
    prophet_rmse = np.sqrt(mean_squared_error(actuals_for_evaluation, prophet_pred))

    # Print results
    print("\n   🤖 TFT Model (evaluated against Prophet's actuals):")
    print(f"      WMAE: ${tft_wmae:,.2f}")
    print(f"      MAE: ${tft_mae:,.2f}")
    print(f"      RMSE: ${tft_rmse:,.2f}")

    print("   📈 Prophet Model:")
    print(f"      WMAE: ${prophet_wmae:,.2f}")
    print(f"      MAE: ${prophet_mae:,.2f}")
    print(f"      RMSE: ${prophet_rmse:,.2f}")

    # Also show Prophet's standalone results for reference
    print("   📈 Prophet Model (Standalone Reference):")
    print(f"      WMAE: ${hybrid_model.prophet_wmae_:,.2f}")

    return {
        'tft_wmae': tft_wmae,
        'tft_mae': tft_mae,
        'tft_rmse': tft_rmse,
        'prophet_wmae': prophet_wmae,
        'prophet_mae': prophet_mae,
        'prophet_rmse': prophet_rmse
    }


In [5]:

def val_step1_load_and_prepare_data():
    """
    Step 1: Load and prepare all data for validation
    Returns: train_full
    """
    print("📊 VALIDATION STEP 1: Loading and preparing data...")
    print("=" * 50)

    # Check if running in Kaggle environment
    if os.path.exists(KAGGLE_DATA_PATH):
        # Kaggle environment
        train_zip_path = os.path.join(KAGGLE_DATA_PATH, 'train.csv.zip')
        features_zip_path = os.path.join(KAGGLE_DATA_PATH, 'features.csv.zip')
        stores_csv_path = os.path.join(KAGGLE_DATA_PATH, 'stores.csv')

        print("   📂 Unzipping necessary data files...")
        with zipfile.ZipFile(train_zip_path, 'r') as zip_ref:
            zip_ref.extractall('.')
        print(f"      - Extracted: {train_zip_path}")

        with zipfile.ZipFile(features_zip_path, 'r') as zip_ref:
            zip_ref.extractall('.')
        print(f"      - Extracted: {features_zip_path}")

        stores_df = pd.read_csv(stores_csv_path)
    else:
        # Local environment - assume files are in current directory
        print("   📂 Loading from local files...")
        if os.path.exists('train.csv.zip'):
            with zipfile.ZipFile('train.csv.zip', 'r') as zip_ref:
                zip_ref.extractall('.')
        if os.path.exists('features.csv.zip'):
            with zipfile.ZipFile('features.csv.zip', 'r') as zip_ref:
                zip_ref.extractall('.')
        stores_df = pd.read_csv('stores.csv')

    # Load the unzipped CSVs
    train_df = pd.read_csv('train.csv')
    features_df = pd.read_csv('features.csv')

    # Convert Date columns to datetime early for consistency
    train_df['Date'] = pd.to_datetime(train_df['Date'])
    features_df['Date'] = pd.to_datetime(features_df['Date'])

    print(f"   📈 Train data: {train_df.shape}")
    print(f"   📊 Features data: {features_df.shape}")
    print(f"   🏪 Stores data: {stores_df.shape}")

    # --- Data Merging and Initial Cleaning ---
    print("\n🧹 Merging data and initial cleaning...")
    merged_df = pd.merge(train_df, features_df, on=['Store', 'Date', 'IsHoliday'], how='left')
    train_full = pd.merge(merged_df, stores_df, on=['Store'], how='left')

    # Fill NaN in MarkDown columns with 0, assuming no markdown if not specified
    markdown_cols = [f'MarkDown{i}' for i in range(1, 6)]
    for col in markdown_cols:
        if col in train_full.columns:
            train_full[col] = train_full[col].fillna(0)

    # Remove rows with negative Weekly_Sales
    initial_rows = len(train_full)
    train_full = train_full[train_full['Weekly_Sales'] > 0]
    print(f"   🗑️ Removed {initial_rows - len(train_full)} rows with negative Weekly_Sales.")

    # Ensure continuous series and fill NaNs for NeuralForecast
    print("   Filling missing dates and sales for time series continuity...")

    # Create a full set of (Store, Dept, Date) unique combinations
    unique_store_dept_dates = train_full[['Store', 'Dept', 'Date']].drop_duplicates()

    # Generate all expected dates for each (Store, Dept)
    df_list = []
    for (store, dept), group in unique_store_dept_dates.groupby(['Store', 'Dept']):
        series_min_date = group['Date'].min()
        series_max_date = group['Date'].max()
        full_series_dates = pd.date_range(start=series_min_date, end=series_max_date, freq='W-FRI')

        temp_df = pd.DataFrame({
            'Store': store,
            'Dept': dept,
            'Date': full_series_dates
        })
        df_list.append(temp_df)

    complete_series_df = pd.concat(df_list, ignore_index=True)

    # Merge the complete series dates with the original train_full data
    train_full_cleaned = pd.merge(
        complete_series_df,
        train_full,
        on=['Store', 'Dept', 'Date'],
        how='left'
    )

    # Fill NaNs in 'Weekly_Sales' with 0
    nan_sales_before_fill = train_full_cleaned['Weekly_Sales'].isnull().sum()
    train_full_cleaned['Weekly_Sales'] = train_full_cleaned['Weekly_Sales'].fillna(0)
    print(f"   Filled {nan_sales_before_fill} NaN Weekly_Sales values with 0 for series continuity.")

    # Re-merge the original features_df and stores_df to fill in associated data
    train_full_cleaned = pd.merge(train_full_cleaned, features_df.drop(columns=['IsHoliday'], errors='ignore'), on=['Store', 'Date'], how='left', suffixes=('', '_feats'))
    train_full_cleaned = pd.merge(train_full_cleaned, stores_df, on=['Store'], how='left', suffixes=('', '_stores'))

    # Combine IsHoliday if it was duplicated
    if 'IsHoliday_feats' in train_full_cleaned.columns:
        train_full_cleaned['IsHoliday'] = train_full_cleaned['IsHoliday'].fillna(train_full_cleaned['IsHoliday_feats'])
        train_full_cleaned = train_full_cleaned.drop(columns=['IsHoliday_feats'])

    # Define numerical columns for filling NaNs
    numerical_cols = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment'] + [f'MarkDown{i}' for i in range(1, 6)]

    # Fill NaNs in features columns
    for col in numerical_cols:
        if col in train_full_cleaned.columns:
            train_full_cleaned[col] = train_full_cleaned.groupby(['Store', 'Dept'])[col].transform(lambda x: x.fillna(x.mean()))
            train_full_cleaned[col] = train_full_cleaned[col].fillna(train_full_cleaned[col].mean())

    # Ensure no negative sales after any filling process
    train_full_cleaned['Weekly_Sales'][train_full_cleaned['Weekly_Sales'] < 0] = 0

    # Sort by date, store, and department for time series consistency
    train_full = train_full_cleaned.sort_values(by=['Date', 'Store', 'Dept']).reset_index(drop=True)

    print(f"   ✅ Merged and cleaned data: {train_full.shape}")
    print(f"   📅 Date range: {train_full['Date'].min()} to {train_full['Date'].max()}")

    print("✅ VALIDATION STEP 1 COMPLETE!")
    return train_full

def val_step2_create_train_val_split(train_full):
    """
    Step 2: Create temporal train/validation split (80/20)
    Returns: X_train, y_train, X_val, y_val
    """
    print("\n📅 VALIDATION STEP 2: Creating temporal split (80/20)...")
    print("=" * 50)

    df_sorted = train_full.sort_values('Date').reset_index(drop=True)

    unique_dates = sorted(df_sorted['Date'].unique())
    total_weeks = len(unique_dates)
    train_ratio = 0.8
    train_weeks = int(total_weeks * train_ratio)

    if train_weeks < 1:
        train_weeks = 1
    if train_weeks >= total_weeks:
        train_weeks = total_weeks - 1

    split_date = unique_dates[train_weeks - 1]

    X_train = df_sorted[df_sorted['Date'] <= split_date].drop(columns=['Weekly_Sales']).copy()
    y_train = df_sorted[df_sorted['Date'] <= split_date]['Weekly_Sales'].copy()
    X_val = df_sorted[df_sorted['Date'] > split_date].drop(columns=['Weekly_Sales']).copy()
    y_val = df_sorted[df_sorted['Date'] > split_date]['Weekly_Sales'].copy()

    print(f"   📊 Split date: {split_date}")
    print(f"   📈 Train: {len(X_train):,} records ({X_train['Date'].min()} to {X_train['Date'].max()})")
    print(f"   📉 Val: {len(X_val):,} records ({X_val['Date'].min()} to {X_val['Date'].max()})")

    print("✅ VALIDATION STEP 2 COMPLETE!")
    return X_train, y_train, X_val, y_val

def val_step3_get_working_tft_predictions():
    """
    Step 3: Get working TFT predictions using the exact working implementation
    Returns: tft_predictions, tft_actuals, tft_holiday_flags, tft_wmae
    """
    print("\n🤖 VALIDATION STEP 3: Getting working TFT predictions...")
    print("=" * 50)

    # Run the exact working TFT implementation
    tft_predictions, tft_actuals, tft_holiday_flags, tft_wmae = run_standalone_tft_experiment()

    print("✅ VALIDATION STEP 3 COMPLETE!")
    print(f"   📊 Generated {len(tft_predictions):,} TFT predictions")
    print(f"   📊 TFT standalone WMAE: ${tft_wmae:,.2f}")

    return tft_predictions, tft_actuals, tft_holiday_flags, tft_wmae

def val_step4_build_tft_pipeline():
    """
    Step 4: Build TFT preprocessing pipeline for validation
    Returns: pipeline
    """
    print("\n🔧 VALIDATION STEP 4: Building TFT preprocessing pipeline...")
    print("=" * 50)

    # Define column lists
    numerical_cols = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment'] + [f'MarkDown{i}' for i in range(1, 6)]
    categorical_ohe_cols = ["Type", "IsHoliday"]
    passthrough_cols = ["Store", "Dept"]

    numerical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean'))
    ])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    preprocessor = ColumnTransformerWithNames(transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_ohe_cols),
        ('pass', 'passthrough', passthrough_cols)
    ], remainder='drop')

    pipeline = Pipeline([
        ("multi_index_keeper", MultiIndexKeeper(index_cols=["Date", "Store", "Dept"])),
        ("date_feature_creator", DateFeatureCreator()),
        ("preprocessor", preprocessor),
        ("tft_regressor", TFTRegressor(
            input_chunk_length=52,
            output_chunk_length=39,
            epochs=25,
            batch_size=32,
            random_seed=42
        ))
    ])

    print("   ✅ Pipeline built successfully")
    print("✅ VALIDATION STEP 4 COMPLETE!")
    return pipeline

def val_step5_train_tft(pipeline, X_train, y_train):
    """
    Step 5: Train TFT model on training data
    FIXED: Use Prophet's exact data preprocessing to match working hybrid model
    Returns: trained pipeline
    """
    print("\n🧠 VALIDATION STEP 5: Training TFT model...")
    print("=" * 50)

    # CRITICAL FIX: Use Prophet's exact data split and preprocessing
    # This matches what the working hybrid model does

    print("   🔄 Getting Prophet's exact data split for TFT training...")

    # Get the same data split Prophet used (exactly like the working hybrid model)
    prophet_pipeline = WalmartProphetPreprocessingPipeline()
    train_full = prophet_pipeline.load_and_prepare_data()
    train_data_raw, val_data_raw, split_info = prophet_pipeline.create_temporal_split(train_full)

    # Extract features and target from Prophet's training data (same as working hybrid)
    X_train_prophet = train_data_raw.drop(['Weekly_Sales'], axis=1)
    y_train_prophet = train_data_raw['Weekly_Sales']

    print(f"   📊 TFT will use Prophet's exact training data: {X_train_prophet.shape}")
    print(f"   📊 Split date: {split_info['split_date']}")

    # Train TFT on Prophet's exact training data through the main pipeline
    pipeline.fit(X_train_prophet, y_train_prophet)

    print("✅ VALIDATION STEP 5 COMPLETE!")
    print("   🎯 TFT model training finished on Prophet's exact data!")
    return pipeline

def val_step6_predict_tft(pipeline, X_val):
    """
    Step 6: Generate TFT predictions on validation data
    FIXED: Use Prophet's exact validation data to match working hybrid model
    Returns: tft_predictions
    """
    print("\n🔮 VALIDATION STEP 6: Generating TFT predictions...")
    print("=" * 50)

    # CRITICAL FIX: Use Prophet's exact validation data
    # This matches what the working hybrid model does

    print("   🔄 Getting Prophet's exact validation data for TFT prediction...")

    # Get the same data split Prophet used (exactly like the working hybrid model)
    prophet_pipeline = WalmartProphetPreprocessingPipeline()
    train_full = prophet_pipeline.load_and_prepare_data()
    train_data_raw, val_data_raw, split_info = prophet_pipeline.create_temporal_split(train_full)

    # Extract validation features from Prophet's validation data (same as working hybrid)
    X_val_prophet = val_data_raw.drop(['Weekly_Sales'], axis=1)

    print(f"   📊 TFT will predict on Prophet's exact validation data: {X_val_prophet.shape}")
    print(f"   📊 Validation period: {val_data_raw['Date'].min()} to {val_data_raw['Date'].max()}")

    # Make TFT predictions on Prophet's exact validation data
    tft_predictions = pipeline.predict(X_val_prophet)

    print("✅ VALIDATION STEP 6 COMPLETE!")
    print(f"   📊 Generated {len(tft_predictions):,} TFT predictions on Prophet's exact validation data!")
    return tft_predictions

def val_step7_create_hybrid_predictions(prophet_predictions, tft_predictions, prophet_holiday_flags):
    """
    Step 7: Create hybrid predictions with adaptive weighting for validation
    Returns: hybrid_predictions, aligned_actuals, aligned_holiday_flags
    """
    print("\n🤝 VALIDATION STEP 7: Creating hybrid predictions...")
    print("=" * 50)

    # Ensure same length
    min_len = min(len(prophet_predictions), len(tft_predictions))
    prophet_pred = prophet_predictions[:min_len]
    tft_pred = tft_predictions[:min_len]
    holiday_flags = prophet_holiday_flags[:min_len]

    print(f"   📊 Aligning predictions: {min_len:,} samples")

    # Apply adaptive weighting
    hybrid_predictions = np.zeros_like(tft_pred, dtype=float)

    # Holiday periods: 60% Prophet + 40% TFT
    holiday_mask = holiday_flags.astype(bool)
    regular_mask = ~holiday_mask

    if np.any(holiday_mask):
        hybrid_predictions[holiday_mask] = (
            0.60 * prophet_pred[holiday_mask] +
            0.40 * tft_pred[holiday_mask]
        )

    # Regular days: 65% TFT + 35% Prophet
    if np.any(regular_mask):
        hybrid_predictions[regular_mask] = (
            0.65 * tft_pred[regular_mask] +
            0.35 * prophet_pred[regular_mask]
        )

    # Ensure no negative predictions
    hybrid_predictions[hybrid_predictions < 0] = 0

    # Report weighting statistics
    n_holiday = np.sum(holiday_mask)
    n_regular = np.sum(regular_mask)
    total_points = len(hybrid_predictions)

    print(f"   📊 Weighting applied:")
    print(f"      Holiday points: {n_holiday:,} ({n_holiday/total_points*100:.1f}%) - 60% Prophet + 40% TFT")
    print(f"      Regular points: {n_regular:,} ({n_regular/total_points*100:.1f}%) - 65% TFT + 35% Prophet")

    print("✅ VALIDATION STEP 7 COMPLETE!")
    return hybrid_predictions, prophet_pred, tft_pred, holiday_flags

def val_step8_evaluate_all_models(hybrid_predictions, prophet_predictions, tft_predictions,
                                  prophet_actuals, prophet_holiday_flags):
    """
    Step 8: Evaluate all models (Prophet, TFT, Hybrid) and compare performance
    Returns: evaluation results dictionary
    """
    print("\n📊 VALIDATION STEP 8: Evaluating all models...")
    print("=" * 50)

    # Use Prophet's actuals and holiday flags for consistent evaluation
    actuals = prophet_actuals
    holiday_flags = prophet_holiday_flags.astype(bool)

    # Ensure all predictions have the same length
    min_len = min(len(hybrid_predictions), len(prophet_predictions), len(tft_predictions), len(actuals))

    hybrid_pred = hybrid_predictions[:min_len]
    prophet_pred = prophet_predictions[:min_len]
    tft_pred = tft_predictions[:min_len]
    actuals_eval = actuals[:min_len]
    holiday_eval = holiday_flags[:min_len]

    print(f"   📊 Evaluating on {min_len:,} samples")

    # Calculate metrics for each model
    results = {}

    # Prophet metrics
    prophet_wmae = calculate_wmae(actuals_eval, prophet_pred, holiday_eval)
    prophet_mae = mean_absolute_error(actuals_eval, prophet_pred)
    prophet_rmse = np.sqrt(mean_squared_error(actuals_eval, prophet_pred))

    # TFT metrics
    tft_wmae = calculate_wmae(actuals_eval, tft_pred, holiday_eval)
    tft_mae = mean_absolute_error(actuals_eval, tft_pred)
    tft_rmse = np.sqrt(mean_squared_error(actuals_eval, tft_pred))

    # Hybrid metrics
    hybrid_wmae = calculate_wmae(actuals_eval, hybrid_pred, holiday_eval)
    hybrid_mae = mean_absolute_error(actuals_eval, hybrid_pred)
    hybrid_rmse = np.sqrt(mean_squared_error(actuals_eval, hybrid_pred))

    # Holiday breakdown
    holiday_mask = holiday_eval
    non_holiday_mask = ~holiday_eval

    # Holiday metrics
    if np.any(holiday_mask):
        prophet_holiday_mae = mean_absolute_error(actuals_eval[holiday_mask], prophet_pred[holiday_mask])
        tft_holiday_mae = mean_absolute_error(actuals_eval[holiday_mask], tft_pred[holiday_mask])
        hybrid_holiday_mae = mean_absolute_error(actuals_eval[holiday_mask], hybrid_pred[holiday_mask])
    else:
        prophet_holiday_mae = tft_holiday_mae = hybrid_holiday_mae = 0

    # Non-holiday metrics
    if np.any(non_holiday_mask):
        prophet_regular_mae = mean_absolute_error(actuals_eval[non_holiday_mask], prophet_pred[non_holiday_mask])
        tft_regular_mae = mean_absolute_error(actuals_eval[non_holiday_mask], tft_pred[non_holiday_mask])
        hybrid_regular_mae = mean_absolute_error(actuals_eval[non_holiday_mask], hybrid_pred[non_holiday_mask])
    else:
        prophet_regular_mae = tft_regular_mae = hybrid_regular_mae = 0

    # Store results
    results = {
        'prophet_wmae': prophet_wmae,
        'prophet_mae': prophet_mae,
        'prophet_rmse': prophet_rmse,
        'tft_wmae': tft_wmae,
        'tft_mae': tft_mae,
        'tft_rmse': tft_rmse,
        'hybrid_wmae': hybrid_wmae,
        'hybrid_mae': hybrid_mae,
        'hybrid_rmse': hybrid_rmse,
        'prophet_holiday_mae': prophet_holiday_mae,
        'tft_holiday_mae': tft_holiday_mae,
        'hybrid_holiday_mae': hybrid_holiday_mae,
        'prophet_regular_mae': prophet_regular_mae,
        'tft_regular_mae': tft_regular_mae,
        'hybrid_regular_mae': hybrid_regular_mae,
        'holiday_samples': np.sum(holiday_mask),
        'regular_samples': np.sum(non_holiday_mask)
    }

    # Print detailed comparison
    print("\n" + "=" * 60)
    print("🎯 VALIDATION RESULTS COMPARISON")
    print("=" * 60)

    print("\n📊 Overall Metrics (Competition Metric = WMAE):")
    print(f"   🏆 Prophet WMAE:  ${prophet_wmae:,.2f}")
    print(f"   🤖 TFT WMAE:      ${tft_wmae:,.2f}")
    print(f"   🤝 Hybrid WMAE:   ${hybrid_wmae:,.2f}")

    print(f"\n📊 MAE Comparison:")
    print(f"   🏆 Prophet MAE:   ${prophet_mae:,.2f}")
    print(f"   🤖 TFT MAE:       ${tft_mae:,.2f}")
    print(f"   🤝 Hybrid MAE:    ${hybrid_mae:,.2f}")

    print(f"\n📊 RMSE Comparison:")
    print(f"   🏆 Prophet RMSE:  ${prophet_rmse:,.2f}")
    print(f"   🤖 TFT RMSE:      ${tft_rmse:,.2f}")
    print(f"   🤝 Hybrid RMSE:   ${hybrid_rmse:,.2f}")

    print(f"\n📊 Holiday Performance ({np.sum(holiday_mask):,} samples):")
    print(f"   🏆 Prophet Holiday MAE:  ${prophet_holiday_mae:,.2f}")
    print(f"   🤖 TFT Holiday MAE:      ${tft_holiday_mae:,.2f}")
    print(f"   🤝 Hybrid Holiday MAE:   ${hybrid_holiday_mae:,.2f}")

    print(f"\n📊 Regular Day Performance ({np.sum(non_holiday_mask):,} samples):")
    print(f"   🏆 Prophet Regular MAE:  ${prophet_regular_mae:,.2f}")
    print(f"   🤖 TFT Regular MAE:      ${tft_regular_mae:,.2f}")
    print(f"   🤝 Hybrid Regular MAE:   ${hybrid_regular_mae:,.2f}")

    # Determine best model
    best_wmae = min(prophet_wmae, tft_wmae, hybrid_wmae)
    if best_wmae == hybrid_wmae:
        best_model = "🤝 Hybrid"
    elif best_wmae == prophet_wmae:
        best_model = "🏆 Prophet"
    else:
        best_model = "🤖 TFT"

    print(f"\n🏆 BEST MODEL: {best_model} (WMAE: ${best_wmae:,.2f})")

    print("✅ VALIDATION STEP 8 COMPLETE!")
    print("🎉 ALL VALIDATION STEPS FINISHED!")

    return results

if __name__ == "__main__":
    # Original main function can still be called if needed
    pass

def run_standalone_tft_experiment():
    """
    Run the exact working TFT experiment from model-exp-tft-fx.ipynb
    This replicates the exact working implementation.

    Returns:
        tuple: (y_pred_val, y_val, is_holiday_val, tft_wmae)
    """
    print("🚀 Running Standalone TFT Experiment (Working Implementation)")
    print("=" * 80)

    try:
        # Step 1: Data Loading (exact copy from working version)
        print("\n📊 Step 1: Data loading...")

        # Check if running in Kaggle environment
        if os.path.exists(KAGGLE_DATA_PATH):
            train_zip_path = os.path.join(KAGGLE_DATA_PATH, 'train.csv.zip')
            features_zip_path = os.path.join(KAGGLE_DATA_PATH, 'features.csv.zip')
            stores_csv_path = os.path.join(KAGGLE_DATA_PATH, 'stores.csv')

            with zipfile.ZipFile(train_zip_path, 'r') as zip_ref:
                zip_ref.extractall('.')
            with zipfile.ZipFile(features_zip_path, 'r') as zip_ref:
                zip_ref.extractall('.')
            stores_df = pd.read_csv(stores_csv_path)
        else:
            # Local environment
            if os.path.exists('train.csv.zip'):
                with zipfile.ZipFile('train.csv.zip', 'r') as zip_ref:
                    zip_ref.extractall('.')
            if os.path.exists('features.csv.zip'):
                with zipfile.ZipFile('features.csv.zip', 'r') as zip_ref:
                    zip_ref.extractall('.')
            stores_df = pd.read_csv('stores.csv')

        # Load datasets
        train_df = pd.read_csv('train.csv')
        features_df = pd.read_csv('features.csv')

        # Convert Date columns to datetime
        train_df['Date'] = pd.to_datetime(train_df['Date'])
        features_df['Date'] = pd.to_datetime(features_df['Date'])

        print(f"   📈 Train data: {train_df.shape}")
        print(f"   📊 Features data: {features_df.shape}")
        print(f"   🏪 Stores data: {stores_df.shape}")

        # Step 2: Data merging and cleaning (exact copy)
        print("\n🧹 Step 2: Data merging and cleaning...")
        merged_df = pd.merge(train_df, features_df, on=['Store', 'Date', 'IsHoliday'], how='left')
        train_full = pd.merge(merged_df, stores_df, on=['Store'], how='left')

        # Fill NaN in MarkDown columns with 0
        markdown_cols = [f'MarkDown{i}' for i in range(1, 6)]
        for col in markdown_cols:
            if col in train_full.columns:
                train_full[col] = train_full[col].fillna(0)

        # Remove rows with negative Weekly_Sales
        initial_rows = len(train_full)
        train_full = train_full[train_full['Weekly_Sales'] > 0]
        print(f"   🗑️ Removed {initial_rows - len(train_full)} rows with negative Weekly_Sales.")

        # Define column lists (exact copy)
        numerical_cols = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment'] + [f'MarkDown{i}' for i in range(1, 6)]
        categorical_ohe_cols = ["Type", "IsHoliday"]
        passthrough_cols = ["Store", "Dept"]

        # Ensure continuous series and fill NaNs (exact copy)
        print("   Filling missing dates and sales for time series continuity...")

        unique_store_dept_dates = train_full[['Store', 'Dept', 'Date']].drop_duplicates()

        df_list = []
        for (store, dept), group in unique_store_dept_dates.groupby(['Store', 'Dept']):
            series_min_date = group['Date'].min()
            series_max_date = group['Date'].max()
            full_series_dates = pd.date_range(start=series_min_date, end=series_max_date, freq='W-FRI')

            temp_df = pd.DataFrame({
                'Store': store,
                'Dept': dept,
                'Date': full_series_dates
            })
            df_list.append(temp_df)

        complete_series_df = pd.concat(df_list, ignore_index=True)

        train_full_cleaned = pd.merge(
            complete_series_df,
            train_full,
            on=['Store', 'Dept', 'Date'],
            how='left'
        )

        nan_sales_before_fill = train_full_cleaned['Weekly_Sales'].isnull().sum()
        train_full_cleaned['Weekly_Sales'] = train_full_cleaned['Weekly_Sales'].fillna(0)
        print(f"   Filled {nan_sales_before_fill} NaN Weekly_Sales values with 0 for series continuity.")

        # Re-merge features and stores data
        train_full_cleaned = pd.merge(train_full_cleaned, features_df.drop(columns=['IsHoliday'], errors='ignore'), on=['Store', 'Date'], how='left', suffixes=('', '_feats'))
        train_full_cleaned = pd.merge(train_full_cleaned, stores_df, on=['Store'], how='left', suffixes=('', '_stores'))

        if 'IsHoliday_feats' in train_full_cleaned.columns:
            train_full_cleaned['IsHoliday'] = train_full_cleaned['IsHoliday'].fillna(train_full_cleaned['IsHoliday_feats'])
            train_full_cleaned = train_full_cleaned.drop(columns=['IsHoliday_feats'])

        # Fill NaNs in features columns
        for col in numerical_cols:
            if col in train_full_cleaned.columns:
                train_full_cleaned[col] = train_full_cleaned.groupby(['Store', 'Dept'])[col].transform(lambda x: x.fillna(x.mean()))
                train_full_cleaned[col] = train_full_cleaned[col].fillna(train_full_cleaned[col].mean())

        train_full_cleaned['Weekly_Sales'][train_full_cleaned['Weekly_Sales'] < 0] = 0
        train_full = train_full_cleaned.sort_values(by=['Date', 'Store', 'Dept']).reset_index(drop=True)

        print(f"   ✅ Merged and cleaned data: {train_full.shape}")
        print(f"   📅 Date range: {train_full['Date'].min()} to {train_full['Date'].max()}")

        # Step 3: Data splitting (80/20 temporal split - exact copy)
        print("\n📅 Step 3: Creating temporal split (80/20)...")

        df_sorted = train_full.sort_values('Date').reset_index(drop=True)
        unique_dates = sorted(df_sorted['Date'].unique())
        total_weeks = len(unique_dates)
        train_ratio = 0.8
        train_weeks = int(total_weeks * train_ratio)

        if train_weeks < 1:
            train_weeks = 1
        if train_weeks >= total_weeks:
            train_weeks = total_weeks - 1

        split_date = unique_dates[train_weeks - 1]

        X_train = df_sorted[df_sorted['Date'] <= split_date].drop(columns=['Weekly_Sales']).copy()
        y_train = df_sorted[df_sorted['Date'] <= split_date]['Weekly_Sales'].copy()
        X_val = df_sorted[df_sorted['Date'] > split_date].drop(columns=['Weekly_Sales']).copy()
        y_val = df_sorted[df_sorted['Date'] > split_date]['Weekly_Sales'].copy()

        print(f"   📊 Split date: {split_date}")
        print(f"   📈 Train: {len(X_train):,} records ({X_train['Date'].min()} to {X_train['Date'].max()})")
        print(f"   📉 Val: {len(X_val):,} records ({X_val['Date'].min()} to {X_val['Date'].max()})")

        # Step 4: Pipeline definition (exact copy)
        print("\n⚙️ Step 4: Defining preprocessing and TFT pipeline...")

        numerical_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='mean'))
        ])

        categorical_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ])

        preprocessor = ColumnTransformerWithNames(transformers=[
            ('num', numerical_transformer, numerical_cols),
            ('cat', categorical_transformer, categorical_ohe_cols),
            ('pass', 'passthrough', passthrough_cols)
        ], remainder='drop')

        pipeline = Pipeline([
            ("multi_index_keeper", MultiIndexKeeper(index_cols=["Date", "Store", "Dept"])),
            ("date_feature_creator", DateFeatureCreator()),
            ("preprocessor", preprocessor),
            ("tft_regressor", TFTRegressor(
                input_chunk_length=52,
                output_chunk_length=39,
                epochs=25,
                batch_size=32,
                random_seed=42
            ))
        ])

        print("   ✅ Pipeline defined.")

        # Step 5: Model training (exact copy)
        print("\n🧠 Step 5: Training TFT model...")
        pipeline.fit(X_train, y_train)
        print("   ✅ TFT Model Training Complete!")

        # Step 6: Model evaluation (exact copy)
        print("\n📊 Step 6: Evaluating model on validation set...")
        y_pred_val = pipeline.predict(X_val)
        y_pred_val[y_pred_val < 0] = 0

        is_holiday_val = X_val['IsHoliday'].values.astype(bool)

        if len(y_val) > 0:
            val_mae = mean_absolute_error(y_val, y_pred_val)
            val_rmse = np.sqrt(mean_squared_error(y_val, y_pred_val))
            val_wmae = calculate_wmae(y_val, y_pred_val, is_holiday_val)
        else:
            val_mae, val_rmse, val_wmae = 0, 0, 0

        holiday_mask_val = is_holiday_val.astype(bool)
        holiday_mae_val = mean_absolute_error(y_val[holiday_mask_val], y_pred_val[holiday_mask_val]) if holiday_mask_val.any() else 0
        non_holiday_mae_val = mean_absolute_error(y_val[~holiday_mask_val], y_pred_val[~holiday_mask_val]) if (~holiday_mask_val).any() else 0

        print("\n" + "=" * 60)
        print("🎯 STANDALONE TFT RESULTS (Working Implementation)")
        print("=" * 60)

        print("\n📊 Validation Metrics:")
        print(f"   WMAE (Competition Metric): ${val_wmae:,.2f}")
        print(f"   MAE: ${val_mae:,.2f}")
        print(f"   RMSE: ${val_rmse:,.2f}")

        print("\n📊 Holiday Breakdown:")
        print(f"   Holiday MAE: ${holiday_mae_val:,.2f} ({int(holiday_mask_val.sum()):,} samples)")
        print(f"   Non-Holiday MAE: ${non_holiday_mae_val:,.2f} ({int((~holiday_mask_val).sum()):,} samples)")

        print("\n🎉 Standalone TFT Experiment: COMPLETE!")

        return y_pred_val, y_val, is_holiday_val, val_wmae

    except Exception as e:
        print(f"❌ Standalone TFT experiment failed: {e}")
        raise

def val_step4_get_prophet_predictions():
    """
    Step 4: Get Prophet predictions for comparison
    Returns: prophet_predictions, prophet_actuals, prophet_holiday_flags, prophet_wmae
    """
    print("\n🏆 VALIDATION STEP 4: Getting Prophet predictions...")
    print("=" * 50)

    # Run standalone Prophet experiment (exact copy from original)
    prophet_predictions, prophet_actuals, prophet_holiday_flags, prophet_wmae = run_standalone_prophet_experiment()

    print("✅ VALIDATION STEP 4 COMPLETE!")
    print(f"   📊 Generated {len(prophet_predictions):,} Prophet predictions")
    print(f"   📊 Prophet standalone WMAE: ${prophet_wmae:,.2f}")

    return prophet_predictions, prophet_actuals, prophet_holiday_flags, prophet_wmae

def val_step5_create_hybrid_predictions(tft_predictions, prophet_predictions, prophet_holiday_flags):
    """
    Step 5: Create hybrid predictions with adaptive weighting for validation
    Returns: hybrid_predictions, aligned_actuals, aligned_holiday_flags
    """
    print("\n🤝 VALIDATION STEP 5: Creating hybrid predictions...")
    print("=" * 50)

    # Ensure same length
    min_len = min(len(tft_predictions), len(prophet_predictions))
    tft_pred = tft_predictions[:min_len]
    prophet_pred = prophet_predictions[:min_len]
    holiday_flags = prophet_holiday_flags[:min_len]

    print(f"   📊 Aligning predictions: {min_len:,} samples")

    # Apply adaptive weighting
    hybrid_predictions = np.zeros_like(tft_pred, dtype=float)

    # Holiday periods: 60% Prophet + 40% TFT
    holiday_mask = holiday_flags.astype(bool)
    regular_mask = ~holiday_mask

    if np.any(holiday_mask):
        hybrid_predictions[holiday_mask] = (
            0.60 * prophet_pred[holiday_mask] +
            0.40 * tft_pred[holiday_mask]
        )

    # Regular days: 65% TFT + 35% Prophet
    if np.any(regular_mask):
        hybrid_predictions[regular_mask] = (
            0.65 * tft_pred[regular_mask] +
            0.35 * prophet_pred[regular_mask]
        )

    # Ensure no negative predictions
    hybrid_predictions[hybrid_predictions < 0] = 0

    # Report weighting statistics
    n_holiday = np.sum(holiday_mask)
    n_regular = np.sum(regular_mask)
    total_points = len(hybrid_predictions)

    print(f"   📊 Weighting applied:")
    print(f"      Holiday points: {n_holiday:,} ({n_holiday/total_points*100:.1f}%) - 60% Prophet + 40% TFT")
    print(f"      Regular points: {n_regular:,} ({n_regular/total_points*100:.1f}%) - 65% TFT + 35% Prophet")

    print("✅ VALIDATION STEP 5 COMPLETE!")
    return hybrid_predictions, tft_pred, prophet_pred, holiday_flags

def val_step6_evaluate_all_models(hybrid_predictions, prophet_predictions, tft_predictions,
                                  prophet_actuals, prophet_holiday_flags):
    """
    Step 6: Evaluate all models (Prophet, TFT, Hybrid) and compare performance
    Returns: evaluation results dictionary
    """
    print("\n📊 VALIDATION STEP 6: Evaluating all models...")
    print("=" * 50)

    # Use Prophet's actuals and holiday flags for consistent evaluation
    actuals = prophet_actuals
    holiday_flags = prophet_holiday_flags.astype(bool)

    # Ensure all predictions have the same length
    min_len = min(len(hybrid_predictions), len(prophet_predictions), len(tft_predictions), len(actuals))

    hybrid_pred = hybrid_predictions[:min_len]
    prophet_pred = prophet_predictions[:min_len]
    tft_pred = tft_predictions[:min_len]
    actuals_eval = actuals[:min_len]
    holiday_eval = holiday_flags[:min_len]

    print(f"   📊 Evaluating on {min_len:,} samples")

    # Calculate metrics for each model
    results = {}

    # Prophet metrics
    prophet_wmae = calculate_wmae(actuals_eval, prophet_pred, holiday_eval)
    prophet_mae = mean_absolute_error(actuals_eval, prophet_pred)
    prophet_rmse = np.sqrt(mean_squared_error(actuals_eval, prophet_pred))

    # TFT metrics
    tft_wmae = calculate_wmae(actuals_eval, tft_pred, holiday_eval)
    tft_mae = mean_absolute_error(actuals_eval, tft_pred)
    tft_rmse = np.sqrt(mean_squared_error(actuals_eval, tft_pred))

    # Hybrid metrics
    hybrid_wmae = calculate_wmae(actuals_eval, hybrid_pred, holiday_eval)
    hybrid_mae = mean_absolute_error(actuals_eval, hybrid_pred)
    hybrid_rmse = np.sqrt(mean_squared_error(actuals_eval, hybrid_pred))

    # Holiday breakdown
    holiday_mask = holiday_eval
    non_holiday_mask = ~holiday_eval

    # Holiday metrics
    if np.any(holiday_mask):
        prophet_holiday_mae = mean_absolute_error(actuals_eval[holiday_mask], prophet_pred[holiday_mask])
        tft_holiday_mae = mean_absolute_error(actuals_eval[holiday_mask], tft_pred[holiday_mask])
        hybrid_holiday_mae = mean_absolute_error(actuals_eval[holiday_mask], hybrid_pred[holiday_mask])
    else:
        prophet_holiday_mae = tft_holiday_mae = hybrid_holiday_mae = 0

    # Non-holiday metrics
    if np.any(non_holiday_mask):
        prophet_regular_mae = mean_absolute_error(actuals_eval[non_holiday_mask], prophet_pred[non_holiday_mask])
        tft_regular_mae = mean_absolute_error(actuals_eval[non_holiday_mask], tft_pred[non_holiday_mask])
        hybrid_regular_mae = mean_absolute_error(actuals_eval[non_holiday_mask], hybrid_pred[non_holiday_mask])
    else:
        prophet_regular_mae = tft_regular_mae = hybrid_regular_mae = 0

    # Store results
    results = {
        'prophet_wmae': prophet_wmae,
        'prophet_mae': prophet_mae,
        'prophet_rmse': prophet_rmse,
        'tft_wmae': tft_wmae,
        'tft_mae': tft_mae,
        'tft_rmse': tft_rmse,
        'hybrid_wmae': hybrid_wmae,
        'hybrid_mae': hybrid_mae,
        'hybrid_rmse': hybrid_rmse,
        'prophet_holiday_mae': prophet_holiday_mae,
        'tft_holiday_mae': tft_holiday_mae,
        'hybrid_holiday_mae': hybrid_holiday_mae,
        'prophet_regular_mae': prophet_regular_mae,
        'tft_regular_mae': tft_regular_mae,
        'hybrid_regular_mae': hybrid_regular_mae,
        'holiday_samples': np.sum(holiday_mask),
        'regular_samples': np.sum(non_holiday_mask)
    }

    # Print detailed comparison
    print("\n" + "=" * 60)
    print("🎯 VALIDATION RESULTS COMPARISON")
    print("=" * 60)

    print("\n📊 Overall Metrics (Competition Metric = WMAE):")
    print(f"   🏆 Prophet WMAE:  ${prophet_wmae:,.2f}")
    print(f"   🤖 TFT WMAE:      ${tft_wmae:,.2f}")
    print(f"   🤝 Hybrid WMAE:   ${hybrid_wmae:,.2f}")

    print(f"\n📊 MAE Comparison:")
    print(f"   🏆 Prophet MAE:   ${prophet_mae:,.2f}")
    print(f"   🤖 TFT MAE:       ${tft_mae:,.2f}")
    print(f"   🤝 Hybrid MAE:    ${hybrid_mae:,.2f}")

    print(f"\n📊 RMSE Comparison:")
    print(f"   🏆 Prophet RMSE:  ${prophet_rmse:,.2f}")
    print(f"   🤖 TFT RMSE:      ${tft_rmse:,.2f}")
    print(f"   🤝 Hybrid RMSE:   ${hybrid_rmse:,.2f}")

    print(f"\n📊 Holiday Performance ({np.sum(holiday_mask):,} samples):")
    print(f"   🏆 Prophet Holiday MAE:  ${prophet_holiday_mae:,.2f}")
    print(f"   🤖 TFT Holiday MAE:      ${tft_holiday_mae:,.2f}")
    print(f"   🤝 Hybrid Holiday MAE:   ${hybrid_holiday_mae:,.2f}")

    print(f"\n📊 Regular Day Performance ({np.sum(non_holiday_mask):,} samples):")
    print(f"   🏆 Prophet Regular MAE:  ${prophet_regular_mae:,.2f}")
    print(f"   🤖 TFT Regular MAE:      ${tft_regular_mae:,.2f}")
    print(f"   🤝 Hybrid Regular MAE:   ${hybrid_regular_mae:,.2f}")

    # Determine best model
    best_wmae = min(prophet_wmae, tft_wmae, hybrid_wmae)
    if best_wmae == hybrid_wmae:
        best_model = "🤝 Hybrid"
    elif best_wmae == prophet_wmae:
        best_model = "🏆 Prophet"
    else:
        best_model = "🤖 TFT"

    print(f"\n🏆 BEST MODEL: {best_model} (WMAE: ${best_wmae:,.2f})")

    print("✅ VALIDATION STEP 6 COMPLETE!")
    print("🎉 ALL VALIDATION STEPS FINISHED!")

    return results


In [6]:
train_full = val_step1_load_and_prepare_data()

📊 VALIDATION STEP 1: Loading and preparing data...
   📂 Loading from local files...
   📈 Train data: (421570, 5)
   📊 Features data: (8190, 12)
   🏪 Stores data: (45, 3)

🧹 Merging data and initial cleaning...
   🗑️ Removed 1358 rows with negative Weekly_Sales.
   Filling missing dates and sales for time series continuity...
   Filled 26507 NaN Weekly_Sales values with 0 for series continuity.
   ✅ Merged and cleaned data: (446719, 27)
   📅 Date range: 2010-02-05 00:00:00 to 2012-10-26 00:00:00
✅ VALIDATION STEP 1 COMPLETE!


In [7]:
X_train, y_train, X_val, y_val = val_step2_create_train_val_split(train_full)


📅 VALIDATION STEP 2: Creating temporal split (80/20)...
   📊 Split date: 2012-04-06 00:00:00
   📈 Train: 357,562 records (2010-02-05 00:00:00 to 2012-04-06 00:00:00)
   📉 Val: 89,157 records (2012-04-13 00:00:00 to 2012-10-26 00:00:00)
✅ VALIDATION STEP 2 COMPLETE!


In [8]:
tft_predictions, tft_actuals, tft_holiday_flags, tft_wmae = val_step3_get_working_tft_predictions()


🤖 VALIDATION STEP 3: Getting working TFT predictions...
🚀 Running Standalone TFT Experiment (Working Implementation)

📊 Step 1: Data loading...
   📈 Train data: (421570, 5)
   📊 Features data: (8190, 12)
   🏪 Stores data: (45, 3)

🧹 Step 2: Data merging and cleaning...
   🗑️ Removed 1358 rows with negative Weekly_Sales.
   Filling missing dates and sales for time series continuity...
   Filled 26507 NaN Weekly_Sales values with 0 for series continuity.
   ✅ Merged and cleaned data: (446719, 27)
   📅 Date range: 2010-02-05 00:00:00 to 2012-10-26 00:00:00

📅 Step 3: Creating temporal split (80/20)...
   📊 Split date: 2012-04-06 00:00:00
   📈 Train: 357,562 records (2010-02-05 00:00:00 to 2012-04-06 00:00:00)
   📉 Val: 89,157 records (2012-04-13 00:00:00 to 2012-10-26 00:00:00)

⚙️ Step 4: Defining preprocessing and TFT pipeline...
   ✅ Pipeline defined.

🧠 Step 5: Training TFT model...


INFO:lightning_fabric.utilities.seed:Seed set to 42
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                    | Type                     | Params | Mode 
-----------------------------------------------------------------------------
0 | loss                    | MAE                      | 0      | train
1 | padder_train            | ConstantPad1d            | 0      | train
2 | scaler                  | TemporalNorm             | 0      | train
3 | embedding               | TFTEmbedding             | 512    | train
4 | temporal_encoder        | TemporalCovariateEncoder | 613 K  | train
5 | temporal_fusion_decoder | TemporalFusionDecoder    | 256 K

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=1000` reached.


   ✅ TFT Model Training Complete!

📊 Step 6: Evaluating model on validation set...
DEBUG: Found NaNs in futr_df_complete for columns: ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'Type_A', 'Type_B', 'Type_C', 'IsHoliday_False', 'IsHoliday_True', 'Store', 'Dept']. Filling with 0.


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

DEBUG: Found NaNs in final y_pred after reindex. Filling with 0.

🎯 STANDALONE TFT RESULTS (Working Implementation)

📊 Validation Metrics:
   WMAE (Competition Metric): $1,420.70
   MAE: $1,595.00
   RMSE: $3,419.24

📊 Holiday Breakdown:
   Holiday MAE: $833.57 (6,617 samples)
   Non-Holiday MAE: $1,656.04 (82,540 samples)

🎉 Standalone TFT Experiment: COMPLETE!
✅ VALIDATION STEP 3 COMPLETE!
   📊 Generated 89,157 TFT predictions
   📊 TFT standalone WMAE: $1,420.70


In [9]:
tft_wmae

np.float64(1420.6975671659154)

In [10]:
prophet_predictions, prophet_actuals, prophet_holiday_flags, prophet_wmae = val_step4_get_prophet_predictions()


🏆 VALIDATION STEP 4: Getting Prophet predictions...
🚀 Running Standalone Prophet Experiment (FX Prophet Exact Copy)

📊 Step 1: Data preprocessing...
🔄 Getting preprocessed data using pipeline...
📊 Loading datasets...
   📈 Train data: (421570, 5)
   📊 Features data: (8190, 12)
   🏪 Stores data: (45, 3)
   ✅ Merged data: (421570, 16)
   📅 Date range: 2010-02-05 00:00:00 to 2012-10-26 00:00:00
📅 Creating temporal split (80/19)...
   📊 Split date: 2012-04-06 00:00:00
   📈 Train: 335,761 records (2010-02-05 00:00:00 to 2012-04-06 00:00:00)
   📉 Val: 85,809 records (2012-04-13 00:00:00 to 2012-10-26 00:00:00)
🔧 Preparing Prophet specific data (holidays)...
✅ Pipeline fitted on training data with holiday-aware settings
🔄 Transforming training data...
✅ Transform complete. Shape: (335761, 5)
🔄 Transforming validation data...
✅ Transform complete. Shape: (85809, 5)
✅ Data preprocessing complete!
   📊 Training shape: (335761, 5)
   📊 Validation shape: (85809, 5)

📈 Step 2: Training Prophet mode

In [11]:
hybrid_predictions, tft_pred, prophet_pred, holiday_flags = val_step5_create_hybrid_predictions(tft_predictions, prophet_predictions, prophet_holiday_flags)


🤝 VALIDATION STEP 5: Creating hybrid predictions...
   📊 Aligning predictions: 85,809 samples
   📊 Weighting applied:
      Holiday points: 2,966 (3.5%) - 60% Prophet + 40% TFT
      Regular points: 82,843 (96.5%) - 65% TFT + 35% Prophet
✅ VALIDATION STEP 5 COMPLETE!


In [13]:
tft_pred

array([26220.701 ,  5215.839 ,  9667.379 , ..., 15219.38  ,  2946.9019,
       43119.54  ], dtype=float32)

In [14]:
len(tft_pred)


85809

In [17]:
tft_pred.shape[0]

85809

In [18]:
prophet_pred

array([8694.93180537, 8072.082305  , 7084.16777916, ...,    0.        ,
          0.        ,    0.        ])

In [19]:
len(prophet_pred)

85809

In [12]:
results = val_step6_evaluate_all_models(hybrid_predictions, prophet_predictions, tft_predictions, prophet_actuals, prophet_holiday_flags)


📊 VALIDATION STEP 6: Evaluating all models...
   📊 Evaluating on 85,809 samples

🎯 VALIDATION RESULTS COMPARISON

📊 Overall Metrics (Competition Metric = WMAE):
   🏆 Prophet WMAE:  $1,599.05
   🤖 TFT WMAE:      $19,889.33
   🤝 Hybrid WMAE:   $12,113.96

📊 MAE Comparison:
   🏆 Prophet MAE:   $1,610.52
   🤖 TFT MAE:       $19,783.11
   🤝 Hybrid MAE:    $12,640.95

📊 RMSE Comparison:
   🏆 Prophet RMSE:  $3,691.86
   🤖 TFT RMSE:      $31,072.15
   🤝 Hybrid RMSE:   $19,925.41

📊 Holiday Performance (2,966 samples):
   🏆 Prophet Holiday MAE:  $1,516.09
   🤖 TFT Holiday MAE:      $20,657.58
   🤝 Hybrid Holiday MAE:   $8,302.40

📊 Regular Day Performance (82,843 samples):
   🏆 Prophet Regular MAE:  $1,613.90
   🤖 TFT Regular MAE:      $19,751.81
   🤝 Hybrid Regular MAE:   $12,796.28

🏆 BEST MODEL: 🏆 Prophet (WMAE: $1,599.05)
✅ VALIDATION STEP 6 COMPLETE!
🎉 ALL VALIDATION STEPS FINISHED!


In [24]:
def correct_hybrid_combination(tft_pred_good, tft_actuals_good, tft_holiday_flags_good,
                               prophet_pred_good, prophet_actuals_good, prophet_holiday_flags_good):
    """
    CORRECT hybrid combination using both models' GOOD predictions on their respective validation sets.
    We'll use Prophet's validation set since that's where both models can be fairly compared.

    The key insight: We need to get TFT predictions on Prophet's exact validation split.

    Args:
        tft_pred_good: TFT predictions (89,157 samples, $1,420 WMAE)
        tft_actuals_good: TFT actuals
        tft_holiday_flags_good: TFT holiday flags
        prophet_pred_good: Prophet predictions (85,809 samples, $1,599 WMAE)
        prophet_actuals_good: Prophet actuals
        prophet_holiday_flags_good: Prophet holiday flags

    Returns:
        Correct evaluation results for all models
    """
    print("\n✅ CORRECT HYBRID COMBINATION: Getting TFT predictions on Prophet's validation set...")
    print("=" * 50)

    print("   🔧 Step 1: Getting Prophet's exact validation split...")

    # Get Prophet's exact data split
    prophet_pipeline = WalmartProphetPreprocessingPipeline()
    train_full = prophet_pipeline.load_and_prepare_data()
    train_data_raw, val_data_raw, split_info = prophet_pipeline.create_temporal_split(train_full)

    print(f"      Prophet's validation: {len(val_data_raw):,} samples")
    print(f"      Split date: {split_info['split_date']}")

    print("   🤖 Step 2: Training TFT on Prophet's exact validation split...")

    # Prepare TFT data using Prophet's exact train/val split
    X_train_tft = train_data_raw.drop(['Weekly_Sales'], axis=1)
    y_train_tft = train_data_raw['Weekly_Sales']
    X_val_tft = val_data_raw.drop(['Weekly_Sales'], axis=1)
    y_val_tft = val_data_raw['Weekly_Sales']

    # Define TFT pipeline (same as working version)
    numerical_cols = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment'] + [f'MarkDown{i}' for i in range(1, 6)]
    categorical_ohe_cols = ["Type", "IsHoliday"]
    passthrough_cols = ["Store", "Dept"]

    numerical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean'))
    ])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    preprocessor = ColumnTransformerWithNames(transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_ohe_cols),
        ('pass', 'passthrough', passthrough_cols)
    ], remainder='drop')

    tft_pipeline = Pipeline([
        ("multi_index_keeper", MultiIndexKeeper(index_cols=["Date", "Store", "Dept"])),
        ("date_feature_creator", DateFeatureCreator()),
        ("preprocessor", preprocessor),
        ("tft_regressor", TFTRegressor(
            input_chunk_length=52,
            output_chunk_length=39,
            epochs=25,
            batch_size=32,
            random_seed=42
        ))
    ])

    # Train TFT on Prophet's exact training split
    print("      Training TFT model on Prophet's training split...")
    tft_pipeline.fit(X_train_tft, y_train_tft)

    # Make TFT predictions on Prophet's exact validation split
    print("      Making TFT predictions on Prophet's validation split...")
    tft_predictions_aligned = tft_pipeline.predict(X_val_tft)
    tft_predictions_aligned[tft_predictions_aligned < 0] = 0

    print("   ✅ Step 3: Now both models are on identical validation data!")

    # Use Prophet's validation data as the reference (both models trained/evaluated on this)
    actuals = prophet_actuals_good
    holiday_flags = prophet_holiday_flags_good
    prophet_pred = prophet_pred_good
    tft_pred = tft_predictions_aligned

    # Ensure same length
    min_len = min(len(tft_pred), len(prophet_pred), len(actuals))
    tft_pred = tft_pred[:min_len]
    prophet_pred = prophet_pred[:min_len]
    actuals = actuals[:min_len]
    holiday_flags = holiday_flags[:min_len]

    print(f"   📊 Evaluation on {min_len:,} identical samples")

    # Verify both models perform well on this validation set
    tft_wmae = calculate_wmae(actuals, tft_pred, holiday_flags)
    prophet_wmae = calculate_wmae(actuals, prophet_pred, holiday_flags)

    print(f"   🔍 Verification - both models should perform well:")
    print(f"      🤖 TFT WMAE on Prophet's validation: ${tft_wmae:,.2f}")
    print(f"      🏆 Prophet WMAE on its validation: ${prophet_wmae:,.2f}")

    # Create hybrid predictions with adaptive weighting
    hybrid_predictions = np.zeros_like(tft_pred, dtype=float)

    holiday_mask = holiday_flags.astype(bool)
    regular_mask = ~holiday_mask

    # Holiday periods: 60% Prophet + 40% TFT
    if np.any(holiday_mask):
        hybrid_predictions[holiday_mask] = (
            0.60 * prophet_pred[holiday_mask] +
            0.40 * tft_pred[holiday_mask]
        )

    # Regular days: 65% TFT + 35% Prophet
    if np.any(regular_mask):
        hybrid_predictions[regular_mask] = (
            0.65 * tft_pred[regular_mask] +
            0.35 * prophet_pred[regular_mask]
        )

    # Ensure no negative predictions
    hybrid_predictions[hybrid_predictions < 0] = 0

    # Calculate all metrics
    hybrid_wmae = calculate_wmae(actuals, hybrid_predictions, holiday_flags)

    tft_mae = mean_absolute_error(actuals, tft_pred)
    prophet_mae = mean_absolute_error(actuals, prophet_pred)
    hybrid_mae = mean_absolute_error(actuals, hybrid_predictions)

    tft_rmse = np.sqrt(mean_squared_error(actuals, tft_pred))
    prophet_rmse = np.sqrt(mean_squared_error(actuals, prophet_pred))
    hybrid_rmse = np.sqrt(mean_squared_error(actuals, hybrid_predictions))

    # Holiday breakdown
    n_holiday = np.sum(holiday_mask)
    n_regular = np.sum(regular_mask)

    if np.any(holiday_mask):
        tft_holiday_mae = mean_absolute_error(actuals[holiday_mask], tft_pred[holiday_mask])
        prophet_holiday_mae = mean_absolute_error(actuals[holiday_mask], prophet_pred[holiday_mask])
        hybrid_holiday_mae = mean_absolute_error(actuals[holiday_mask], hybrid_predictions[holiday_mask])
    else:
        tft_holiday_mae = prophet_holiday_mae = hybrid_holiday_mae = 0

    if np.any(regular_mask):
        tft_regular_mae = mean_absolute_error(actuals[regular_mask], tft_pred[regular_mask])
        prophet_regular_mae = mean_absolute_error(actuals[regular_mask], prophet_pred[regular_mask])
        hybrid_regular_mae = mean_absolute_error(actuals[regular_mask], hybrid_predictions[regular_mask])
    else:
        tft_regular_mae = prophet_regular_mae = hybrid_regular_mae = 0

    # Print final results
    print(f"\n   📊 Hybrid weighting applied:")
    print(f"      Holiday points: {n_holiday:,} ({n_holiday/min_len*100:.1f}%) - 60% Prophet + 40% TFT")
    print(f"      Regular points: {n_regular:,} ({n_regular/min_len*100:.1f}%) - 65% TFT + 35% Prophet")

    print("\n" + "=" * 60)
    print("🎯 CORRECT HYBRID RESULTS (Both Models Performing Well)")
    print("=" * 60)

    print("\n📊 Overall Metrics (Competition Metric = WMAE):")
    print(f"   🤖 TFT WMAE:      ${tft_wmae:,.2f}")
    print(f"   🏆 Prophet WMAE:  ${prophet_wmae:,.2f}")
    print(f"   🤝 Hybrid WMAE:   ${hybrid_wmae:,.2f}")

    print(f"\n📊 MAE Comparison:")
    print(f"   🤖 TFT MAE:       ${tft_mae:,.2f}")
    print(f"   🏆 Prophet MAE:   ${prophet_mae:,.2f}")
    print(f"   🤝 Hybrid MAE:    ${hybrid_mae:,.2f}")

    print(f"\n📊 RMSE Comparison:")
    print(f"   🤖 TFT RMSE:      ${tft_rmse:,.2f}")
    print(f"   🏆 Prophet RMSE:  ${prophet_rmse:,.2f}")
    print(f"   🤝 Hybrid RMSE:   ${hybrid_rmse:,.2f}")

    print(f"\n📊 Holiday Performance ({n_holiday:,} samples):")
    print(f"   🤖 TFT Holiday MAE:      ${tft_holiday_mae:,.2f}")
    print(f"   🏆 Prophet Holiday MAE:  ${prophet_holiday_mae:,.2f}")
    print(f"   🤝 Hybrid Holiday MAE:   ${hybrid_holiday_mae:,.2f}")

    print(f"\n📊 Regular Day Performance ({n_regular:,} samples):")
    print(f"   🤖 TFT Regular MAE:      ${tft_regular_mae:,.2f}")
    print(f"   🏆 Prophet Regular MAE:  ${prophet_regular_mae:,.2f}")
    print(f"   🤝 Hybrid Regular MAE:   ${hybrid_regular_mae:,.2f}")

    # Determine best model
    best_wmae = min(tft_wmae, prophet_wmae, hybrid_wmae)
    if best_wmae == hybrid_wmae:
        best_model = "🤝 Hybrid"
    elif best_wmae == tft_wmae:
        best_model = "🤖 TFT"
    else:
        best_model = "🏆 Prophet"

    print(f"\n🏆 BEST MODEL: {best_model} (WMAE: ${best_wmae:,.2f})")
    print("✅ CORRECT HYBRID COMBINATION COMPLETE!")

    return {
        'tft_wmae': tft_wmae,
        'prophet_wmae': prophet_wmae,
        'hybrid_wmae': hybrid_wmae,
        'best_model': best_model,
        'best_wmae': best_wmae
    }


In [25]:
# This will train TFT on Prophet's exact validation split so both models perform well
results = correct_hybrid_combination(
    tft_predictions,        # Your good TFT predictions (89,157 samples)
    tft_actuals,           # TFT actuals
    tft_holiday_flags,     # TFT holiday flags
    prophet_predictions,   # Prophet predictions (85,809 samples, $1,599 WMAE)
    prophet_actuals,       # Prophet actuals
    prophet_holiday_flags  # Prophet holiday flags
)


✅ CORRECT HYBRID COMBINATION: Getting TFT predictions on Prophet's validation set...
   🔧 Step 1: Getting Prophet's exact validation split...
📊 Loading datasets...
   📈 Train data: (421570, 5)
   📊 Features data: (8190, 12)
   🏪 Stores data: (45, 3)
   ✅ Merged data: (421570, 16)
   📅 Date range: 2010-02-05 00:00:00 to 2012-10-26 00:00:00
📅 Creating temporal split (80/19)...
   📊 Split date: 2012-04-06 00:00:00
   📈 Train: 335,761 records (2010-02-05 00:00:00 to 2012-04-06 00:00:00)
   📉 Val: 85,809 records (2012-04-13 00:00:00 to 2012-10-26 00:00:00)
      Prophet's validation: 85,809 samples
      Split date: 2012-04-06 00:00:00
   🤖 Step 2: Training TFT on Prophet's exact validation split...
      Training TFT model on Prophet's training split...


INFO:lightning_fabric.utilities.seed:Seed set to 42
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                    | Type                     | Params | Mode 
-----------------------------------------------------------------------------
0 | loss                    | MAE                      | 0      | train
1 | padder_train            | ConstantPad1d            | 0      | train
2 | scaler                  | TemporalNorm             | 0      | train
3 | embedding               | TFTEmbedding             | 512    | train
4 | temporal_encoder        | TemporalCovariateEncoder | 613 K  | train
5 | temporal_fusion_decoder | TemporalFusionDecoder    | 256 K

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=1000` reached.


      Making TFT predictions on Prophet's validation split...
DEBUG: Found NaNs in futr_df_complete for columns: ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'Type_A', 'Type_B', 'Type_C', 'IsHoliday_False', 'IsHoliday_True', 'Store', 'Dept']. Filling with 0.


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

DEBUG: Found NaNs in final y_pred after reindex. Filling with 0.
   ✅ Step 3: Now both models are on identical validation data!
   📊 Evaluation on 85,809 identical samples
   🔍 Verification - both models should perform well:
      🤖 TFT WMAE on Prophet's validation: $19,774.59
      🏆 Prophet WMAE on its validation: $1,599.05

   📊 Hybrid weighting applied:
      Holiday points: 2,966 (3.5%) - 60% Prophet + 40% TFT
      Regular points: 82,843 (96.5%) - 65% TFT + 35% Prophet

🎯 CORRECT HYBRID RESULTS (Both Models Performing Well)

📊 Overall Metrics (Competition Metric = WMAE):
   🤖 TFT WMAE:      $19,774.59
   🏆 Prophet WMAE:  $1,599.05
   🤝 Hybrid WMAE:   $12,044.32

📊 MAE Comparison:
   🤖 TFT MAE:       $19,625.80
   🏆 Prophet MAE:   $1,610.52
   🤝 Hybrid MAE:    $12,544.64

📊 RMSE Comparison:
   🤖 TFT RMSE:      $30,895.52
   🏆 Prophet RMSE:  $3,691.86
   🤝 Hybrid RMSE:   $19,814.32

📊 Holiday Performance (2,966 samples):
   🤖 TFT Holiday MAE:      $20,850.75
   🏆 Prophet Holiday MA

In [26]:

def simple_alignment_hybrid(tft_pred_good, tft_actuals_good, tft_holiday_flags_good,
                            prophet_pred_good, prophet_actuals_good, prophet_holiday_flags_good):
    """
    Simple alignment: Take the LAST 85,809 TFT predictions to match Prophet's validation set.
    This is the most straightforward approach since both models should cover overlapping time periods.

    Args:
        tft_pred_good: TFT predictions (89,157 samples)
        tft_actuals_good: TFT actuals (89,157 samples)
        tft_holiday_flags_good: TFT holiday flags (89,157 samples)
        prophet_pred_good: Prophet predictions (85,809 samples)
        prophet_actuals_good: Prophet actuals (85,809 samples)
        prophet_holiday_flags_good: Prophet holiday flags (85,809 samples)

    Returns:
        Evaluation results for all models
    """
    print("\n🎯 SIMPLE ALIGNMENT HYBRID: Using last 85,809 TFT predictions...")
    print("=" * 50)

    prophet_len = len(prophet_pred_good)  # 85,809
    tft_len = len(tft_pred_good)         # 89,157

    print(f"   📊 Input data:")
    print(f"      TFT predictions: {tft_len:,} samples")
    print(f"      Prophet predictions: {prophet_len:,} samples")
    print(f"      Taking LAST {prophet_len:,} TFT predictions for alignment")

    # Take the LAST 85,809 TFT predictions to match Prophet's validation period
    tft_pred = tft_pred_good[-prophet_len:]
    tft_actuals = tft_actuals_good[-prophet_len:]
    tft_holidays = tft_holiday_flags_good[-prophet_len:]

    # Use Prophet's predictions as-is
    prophet_pred = prophet_pred_good
    prophet_actuals = prophet_actuals_good
    prophet_holidays = prophet_holiday_flags_good

    print(f"   ✅ Aligned to {prophet_len:,} samples")
    print(f"   📅 Using Prophet's validation period as reference")

    # Verify lengths match
    assert len(tft_pred) == len(prophet_pred) == len(prophet_actuals) == prophet_len

    # Use Prophet's actuals as the ground truth (since Prophet's validation set is well-defined)
    actuals = prophet_actuals
    holiday_flags = prophet_holidays

    # Create hybrid predictions with adaptive weighting
    hybrid_predictions = np.zeros_like(tft_pred, dtype=float)

    holiday_mask = holiday_flags.astype(bool)
    regular_mask = ~holiday_mask

    # Holiday periods: 60% Prophet + 40% TFT
    if np.any(holiday_mask):
        hybrid_predictions[holiday_mask] = (
            0.60 * prophet_pred[holiday_mask] +
            0.40 * tft_pred[holiday_mask]
        )

    # Regular days: 65% TFT + 35% Prophet
    if np.any(regular_mask):
        hybrid_predictions[regular_mask] = (
            0.65 * tft_pred[regular_mask] +
            0.35 * prophet_pred[regular_mask]
        )

    # Ensure no negative predictions
    hybrid_predictions[hybrid_predictions < 0] = 0

    # Calculate metrics (all evaluated against Prophet's actuals)
    tft_wmae = calculate_wmae(actuals, tft_pred, holiday_flags)
    prophet_wmae = calculate_wmae(actuals, prophet_pred, holiday_flags)
    hybrid_wmae = calculate_wmae(actuals, hybrid_predictions, holiday_flags)

    tft_mae = mean_absolute_error(actuals, tft_pred)
    prophet_mae = mean_absolute_error(actuals, prophet_pred)
    hybrid_mae = mean_absolute_error(actuals, hybrid_predictions)

    tft_rmse = np.sqrt(mean_squared_error(actuals, tft_pred))
    prophet_rmse = np.sqrt(mean_squared_error(actuals, prophet_pred))
    hybrid_rmse = np.sqrt(mean_squared_error(actuals, hybrid_predictions))

    # Holiday breakdown
    n_holiday = np.sum(holiday_mask)
    n_regular = np.sum(regular_mask)

    if np.any(holiday_mask):
        tft_holiday_mae = mean_absolute_error(actuals[holiday_mask], tft_pred[holiday_mask])
        prophet_holiday_mae = mean_absolute_error(actuals[holiday_mask], prophet_pred[holiday_mask])
        hybrid_holiday_mae = mean_absolute_error(actuals[holiday_mask], hybrid_predictions[holiday_mask])
    else:
        tft_holiday_mae = prophet_holiday_mae = hybrid_holiday_mae = 0

    if np.any(regular_mask):
        tft_regular_mae = mean_absolute_error(actuals[regular_mask], tft_pred[regular_mask])
        prophet_regular_mae = mean_absolute_error(actuals[regular_mask], prophet_pred[regular_mask])
        hybrid_regular_mae = mean_absolute_error(actuals[regular_mask], hybrid_predictions[regular_mask])
    else:
        tft_regular_mae = prophet_regular_mae = hybrid_regular_mae = 0

    # Print results
    print(f"\n   📊 Hybrid weighting applied:")
    print(f"      Holiday points: {n_holiday:,} ({n_holiday/prophet_len*100:.1f}%) - 60% Prophet + 40% TFT")
    print(f"      Regular points: {n_regular:,} ({n_regular/prophet_len*100:.1f}%) - 65% TFT + 35% Prophet")

    print("\n" + "=" * 60)
    print("🎯 SIMPLE ALIGNMENT HYBRID RESULTS")
    print("=" * 60)

    print("\n📊 Overall Metrics (Competition Metric = WMAE):")
    print(f"   🤖 TFT WMAE:      ${tft_wmae:,.2f}")
    print(f"   🏆 Prophet WMAE:  ${prophet_wmae:,.2f}")
    print(f"   🤝 Hybrid WMAE:   ${hybrid_wmae:,.2f}")

    print(f"\n📊 MAE Comparison:")
    print(f"   🤖 TFT MAE:       ${tft_mae:,.2f}")
    print(f"   🏆 Prophet MAE:   ${prophet_mae:,.2f}")
    print(f"   🤝 Hybrid MAE:    ${hybrid_mae:,.2f}")

    print(f"\n📊 RMSE Comparison:")
    print(f"   🤖 TFT RMSE:      ${tft_rmse:,.2f}")
    print(f"   🏆 Prophet RMSE:  ${prophet_rmse:,.2f}")
    print(f"   🤝 Hybrid RMSE:   ${hybrid_rmse:,.2f}")

    print(f"\n📊 Holiday Performance ({n_holiday:,} samples):")
    print(f"   🤖 TFT Holiday MAE:      ${tft_holiday_mae:,.2f}")
    print(f"   🏆 Prophet Holiday MAE:  ${prophet_holiday_mae:,.2f}")
    print(f"   🤝 Hybrid Holiday MAE:   ${hybrid_holiday_mae:,.2f}")

    print(f"\n📊 Regular Day Performance ({n_regular:,} samples):")
    print(f"   🤖 TFT Regular MAE:      ${tft_regular_mae:,.2f}")
    print(f"   🏆 Prophet Regular MAE:  ${prophet_regular_mae:,.2f}")
    print(f"   🤝 Hybrid Regular MAE:   ${hybrid_regular_mae:,.2f}")

    # Determine best model
    best_wmae = min(tft_wmae, prophet_wmae, hybrid_wmae)
    if best_wmae == hybrid_wmae:
        best_model = "🤝 Hybrid"
    elif best_wmae == tft_wmae:
        best_model = "🤖 TFT"
    else:
        best_model = "🏆 Prophet"

    print(f"\n🏆 BEST MODEL: {best_model} (WMAE: ${best_wmae:,.2f})")
    print("✅ SIMPLE ALIGNMENT HYBRID COMPLETE!")

    return {
        'tft_wmae': tft_wmae,
        'prophet_wmae': prophet_wmae,
        'hybrid_wmae': hybrid_wmae,
        'best_model': best_model,
        'best_wmae': best_wmae
    }


In [27]:
# Use the last 85,809 TFT predictions to match Prophet's validation set exactly
results = simple_alignment_hybrid(
    tft_predictions,        # Your good TFT predictions (89,157 samples)
    tft_actuals,           # TFT actuals (89,157 samples)
    tft_holiday_flags,     # TFT holiday flags (89,157 samples)
    prophet_predictions,   # Prophet predictions (85,809 samples)
    prophet_actuals,       # Prophet actuals (85,809 samples)
    prophet_holiday_flags  # Prophet holiday flags (85,809 samples)
)


🎯 SIMPLE ALIGNMENT HYBRID: Using last 85,809 TFT predictions...
   📊 Input data:
      TFT predictions: 89,157 samples
      Prophet predictions: 85,809 samples
      Taking LAST 85,809 TFT predictions for alignment
   ✅ Aligned to 85,809 samples
   📅 Using Prophet's validation period as reference

   📊 Hybrid weighting applied:
      Holiday points: 2,966 (3.5%) - 60% Prophet + 40% TFT
      Regular points: 82,843 (96.5%) - 65% TFT + 35% Prophet

🎯 SIMPLE ALIGNMENT HYBRID RESULTS

📊 Overall Metrics (Competition Metric = WMAE):
   🤖 TFT WMAE:      $19,920.92
   🏆 Prophet WMAE:  $1,599.05
   🤝 Hybrid WMAE:   $12,150.25

📊 MAE Comparison:
   🤖 TFT MAE:       $19,848.67
   🏆 Prophet MAE:   $1,610.52
   🤝 Hybrid MAE:    $12,692.93

📊 RMSE Comparison:
   🤖 TFT RMSE:      $31,181.34
   🏆 Prophet RMSE:  $3,691.86
   🤝 Hybrid RMSE:   $20,004.32

📊 Holiday Performance (2,966 samples):
   🤖 TFT Holiday MAE:      $20,443.50
   🏆 Prophet Holiday MAE:  $1,516.09
   🤝 Hybrid Holiday MAE:   $8,225.2

In [29]:
#!/usr/bin/env python3
"""
Walmart Sales Forecasting - TFT Only Inference
Trains TFT model on full training dataset and generates submission for test.csv
"""

import warnings
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error
from neuralforecast import NeuralForecast
from neuralforecast.models import TFT
import zipfile
import os
import logging
from datetime import datetime, timedelta

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")
logging.getLogger('pytorch_lightning').setLevel(logging.WARNING)
logging.getLogger('lightning_fabric').setLevel(logging.WARNING)
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)

# Data path configuration
KAGGLE_DATA_PATH = "/kaggle/input/walmart-recruiting-store-sales-forecasting/"

# =============================================================================
# TFT Components (from working implementation)
# =============================================================================

class DateFeatureCreator(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        if "Date" not in X.columns:
            raise ValueError("DateFeatureCreator requires 'Date' column in input X.")

        # Ensure 'Date' is datetime type before operations
        if not pd.api.types.is_datetime64_any_dtype(X['Date']):
            X['Date'] = pd.to_datetime(X['Date'])

        # Using to_period('W') and then converting to integer week number
        # rank(method="dense") ensures consecutive integers for weeks
        X["week"] = (X["Date"].dt.to_period("W").rank(method="dense").astype(int) - 1)

        # Cyclical features for different periodicities
        X["sin_13"] = np.sin(2 * np.pi * X["week"] / 13) # Roughly quarterly seasonality
        X["cos_13"] = np.cos(2 * np.pi * X["week"] / 13)
        X["sin_23"] = np.sin(2 * np.pi * X["week"] / 23) # A different, less common periodicity
        X["cos_23"] = np.cos(2 * np.pi * X["week"] / 23)

        # Drop the original 'Date' column as its information is now in cyclical features
        X = X.drop(columns=["Date"], errors='ignore')
        return X

class ColumnTransformerWithNames(ColumnTransformer):
    """
    A wrapper around ColumnTransformer to retain column names and return a DataFrame.
    Handles OneHotEncoder output specifically.
    """
    def __init__(self, transformers, remainder='drop'):
        super().__init__(transformers=transformers, remainder=remainder)
        self.output_columns_ = None

    def fit(self, X, y=None):
        super().fit(X, y)
        self.output_columns_ = self._get_feature_names_out_internal(X)
        return self

    def _get_feature_names_out_internal(self, X):
        column_names = []
        for name, transformer, columns in self.transformers_:
            if transformer == 'drop':
                continue
            elif transformer == 'passthrough':
                # Ensure passthrough columns are correctly identified from original X
                if isinstance(columns, str):
                    column_names.append(columns)
                else:
                    column_names.extend(list(columns))
            else:
                if hasattr(transformer, 'get_feature_names_out'):
                    if isinstance(columns, str):
                        col_names = [columns]
                    else:
                        col_names = list(columns)
                    column_names.extend(list(transformer.get_feature_names_out(col_names)))
                else:
                    if isinstance(columns, str): # Fallback for transformers without get_feature_names_out
                        column_names.append(columns)
                    else:
                        column_names.extend(list(columns))
        return column_names

    def transform(self, X):
        transformed_array = super().transform(X)
        if self.output_columns_ is None:
             raise RuntimeError("ColumnTransformerWithNames must be fitted before transform.")

        # Convert to dense array if it's a sparse matrix (older sklearn default for OHE)
        if hasattr(transformed_array, 'toarray'):
            transformed_array = transformed_array.toarray()

        # Ensure that the index is preserved from the input X
        # This is CRITICAL for maintaining alignment with y
        return pd.DataFrame(transformed_array, index=X.index, columns=self.output_columns_)

    def fit_transform(self, X, y=None):
        transformed_array = super().fit_transform(X, y)
        self.output_columns_ = self._get_feature_names_out_internal(X)

        # Convert to dense array if it's a sparse matrix (older sklearn default for OHE)
        if hasattr(transformed_array, 'toarray'):
            transformed_array = transformed_array.toarray()

        # Ensure that the index is preserved from the input X
        # This is CRITICAL for maintaining alignment with y
        return pd.DataFrame(transformed_array, index=X.index, columns=self.output_columns_)

class MultiIndexKeeper(BaseEstimator, TransformerMixin):
    def __init__(self, index_cols=["Date", "Store", "Dept"]):
        self.index_cols = index_cols

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()

        if 'Date' in X.columns and not pd.api.types.is_datetime64_any_dtype(X['Date']):
            X['Date'] = pd.to_datetime(X['Date'])

        missing_cols = [col for col in self.index_cols if col not in X.columns]
        if missing_cols:
            raise ValueError(f"MultiIndexKeeper: Missing columns in input X: {missing_cols}")

        # IMPORTANT: When setting index, ensure 'Date', 'Store', 'Dept' columns
        # are not dropped, as they are needed later by NeuralForecast as covariates.
        # This is already handled by drop=False.
        X.set_index(self.index_cols, drop=False, inplace=True)
        return X

class TFTRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, input_chunk_length=52, output_chunk_length=39, epochs=25, batch_size=32, random_seed=42):
        self.input_chunk_length = input_chunk_length
        self.output_chunk_length = output_chunk_length
        self.epochs = epochs
        self.batch_size = batch_size
        self.random_seed = random_seed
        self.nf_ = None
        self.model_ = None
        self.trained_df_ = None # Store the DataFrame used for training

    def fit(self, X, y):
        # Ensure y has a name for proper merging if it doesn't already
        if y.name is None:
            y.name = 'y'

        y_multiindexed = pd.Series(y.values, index=X.index, name='y')

        df = X.copy()
        df['y'] = y_multiindexed

        if not pd.api.types.is_datetime64_any_dtype(df.index.get_level_values('Date')):
            raise ValueError("MultiIndex 'Date' level is not datetime type. Ensure MultiIndexKeeper makes it datetime.")
        df['ds'] = df.index.get_level_values('Date')

        df["unique_id"] = df["Store"].astype(str) + "_" + df["Dept"].astype(str)

        # Store the prepared DataFrame for use in predict
        self.trained_df_ = df.copy() # Store the full df (including y and features)

        # --- DEBUGGING STEP: Check for NaNs immediately before NeuralForecast fit ---
        nan_check = df.isnull().sum()
        cols_with_nans = nan_check[nan_check > 0].index.tolist()
        if cols_with_nans:
            print(f"DEBUG: Found NaNs in the following columns before NeuralForecast fit: {cols_with_nans}")
            # Print head including index for better context
            print(df.loc[df[cols_with_nans[0]].isnull(), cols_with_nans].head())
            raise ValueError(f"Found missing values in {cols_with_nans}.")
        # --- END DEBUGGING STEP ---

        self.model_ = TFT(
            h=self.output_chunk_length,
            input_size=self.input_chunk_length,
            batch_size=self.batch_size,
            random_seed=self.random_seed,
        )

        self.nf_ = NeuralForecast(models=[self.model_], freq="W-FRI")

        self.nf_.fit(df=df)
        return self

    def predict(self, X):
        # X here is the transformed X_test from the pipeline, with MultiIndex ('Date', 'Store', 'Dept')

        # 1. Prepare future covariates from X (which is X_test after preprocessing)
        df_future_covariates_raw = X.copy() # This X contains all features for the test period

        if not pd.api.types.is_datetime64_any_dtype(df_future_covariates_raw.index.get_level_values('Date')):
            raise ValueError("MultiIndex 'Date' level is not datetime type in predict. Ensure MultiIndexKeeper makes it datetime.")

        df_future_covariates_raw['ds'] = df_future_covariates_raw.index.get_level_values('Date')
        df_future_covariates_raw["unique_id"] = df_future_covariates_raw["Store"].astype(str) + "_" + df_future_covariates_raw["Dept"].astype(str)

        # Identify all covariate columns (all columns in self.trained_df_ except 'ds', 'unique_id', 'y')
        covariate_cols = [col for col in self.trained_df_.columns if col not in ['ds', 'unique_id', 'y']]

        # Select only the relevant future covariate columns and the required 'ds', 'unique_id'
        df_future_covariates_selected = df_future_covariates_raw[['ds', 'unique_id'] + covariate_cols].copy()

        # 2. Generate the full expected future dataframe for all series for the forecast horizon
        expected_future_df_template = self.nf_.make_future_dataframe(self.trained_df_)

        # 3. Merge the generated template with our actual future covariates (X_test)
        futr_df_complete = pd.merge(
            expected_future_df_template,
            df_future_covariates_selected,
            on=['unique_id', 'ds'],
            how='left'
        )

        nan_check_futr = futr_df_complete.isnull().sum()
        cols_with_nans_futr = nan_check_futr[nan_check_futr > 0].index.tolist()
        if cols_with_nans_futr:
            print(f"DEBUG: Found NaNs in futr_df_complete for columns: {cols_with_nans_futr}. Filling with 0.")
            futr_df_complete[cols_with_nans_futr] = futr_df_complete[cols_with_nans_futr].fillna(0)

        # 4. Perform the prediction
        # Ensure that `df` has all required unique_ids and `futr_df` aligns.
        # This is the point where the model generates predictions.
        forecast_df = self.nf_.predict(df=self.trained_df_, futr_df=futr_df_complete)

        forecast_df = forecast_df.rename(columns={'TFT': 'yhat'})

        if 'unique_id' in forecast_df.columns:
            forecast_df[['Store', 'Dept']] = forecast_df['unique_id'].str.split('_', expand=True)
            forecast_df['Store'] = forecast_df['Store'].astype(float).astype(int)
            forecast_df['Dept'] = forecast_df['Dept'].astype(float).astype(int)

        forecast_df['Date'] = pd.to_datetime(forecast_df['ds'])

        # Ensure the index columns are correct before setting the index
        # Also, make sure that 'Store' and 'Dept' are properly integer type before setting multi-index
        forecast_df_indexed = forecast_df.set_index(['Date', 'Store', 'Dept'])[['yhat']]

        # This is where NaNs can be introduced if forecast_df_indexed doesn't cover all X.index
        final_predictions = forecast_df_indexed.reindex(X.index)

        y_pred = final_predictions['yhat'].values.flatten()

        # FIX: Fill any NaNs in the final predictions array with 0 before evaluation
        if np.isnan(y_pred).any():
            print("DEBUG: Found NaNs in final y_pred after reindex. Filling with 0.")
            y_pred = np.nan_to_num(y_pred, nan=0.0)

        y_pred[y_pred < 0] = 0

        return y_pred


In [30]:

def step1_load_and_prepare_data():
    """
    Step 1: Load and prepare training and test data
    Returns: train_full, test_full
    """
    print("📊 STEP 1: Loading and preparing data...")
    print("=" * 50)

    # Check if running in Kaggle environment
    if os.path.exists(KAGGLE_DATA_PATH):
        train_zip_path = os.path.join(KAGGLE_DATA_PATH, 'train.csv.zip')
        features_zip_path = os.path.join(KAGGLE_DATA_PATH, 'features.csv.zip')
        test_zip_path = os.path.join(KAGGLE_DATA_PATH, 'test.csv.zip')
        stores_csv_path = os.path.join(KAGGLE_DATA_PATH, 'stores.csv')

        print("   📂 Unzipping data files...")
        with zipfile.ZipFile(train_zip_path, 'r') as zip_ref:
            zip_ref.extractall('.')
        with zipfile.ZipFile(features_zip_path, 'r') as zip_ref:
            zip_ref.extractall('.')
        with zipfile.ZipFile(test_zip_path, 'r') as zip_ref:
            zip_ref.extractall('.')
        stores_df = pd.read_csv(stores_csv_path)
    else:
        print("   📂 Loading from local files...")
        if os.path.exists('train.csv.zip'):
            with zipfile.ZipFile('train.csv.zip', 'r') as zip_ref:
                zip_ref.extractall('.')
        if os.path.exists('features.csv.zip'):
            with zipfile.ZipFile('features.csv.zip', 'r') as zip_ref:
                zip_ref.extractall('.')
        if os.path.exists('test.csv.zip'):
            with zipfile.ZipFile('test.csv.zip', 'r') as zip_ref:
                zip_ref.extractall('.')
        stores_df = pd.read_csv('stores.csv')

    # Load datasets
    train_df = pd.read_csv('train.csv')
    test_df = pd.read_csv('test.csv')
    features_df = pd.read_csv('features.csv')

    # Convert Date columns to datetime
    train_df['Date'] = pd.to_datetime(train_df['Date'])
    test_df['Date'] = pd.to_datetime(test_df['Date'])
    features_df['Date'] = pd.to_datetime(features_df['Date'])

    print(f"   📈 Train data: {train_df.shape}")
    print(f"   📉 Test data: {test_df.shape}")
    print(f"   📊 Features data: {features_df.shape}")
    print(f"   🏪 Stores data: {stores_df.shape}")

    # Merge training data
    print("   🧹 Merging training data and initial cleaning...")
    merged_train = pd.merge(train_df, features_df, on=['Store', 'Date', 'IsHoliday'], how='left')
    train_full = pd.merge(merged_train, stores_df, on=['Store'], how='left')

    # Merge test data
    print("   🧹 Merging test data...")
    merged_test = pd.merge(test_df, features_df, on=['Store', 'Date'], how='left')
    test_full = pd.merge(merged_test, stores_df, on=['Store'], how='left')

    # Handle markdown columns
    markdown_cols = [f'MarkDown{i}' for i in range(1, 6)]
    for col in markdown_cols:
        if col in train_full.columns:
            train_full[col] = train_full[col].fillna(0)
        if col in test_full.columns:
            test_full[col] = test_full[col].fillna(0)

    # Remove negative sales from training
    initial_rows = len(train_full)
    train_full = train_full[train_full['Weekly_Sales'] > 0]
    print(f"   🗑️ Removed {initial_rows - len(train_full)} rows with negative Weekly_Sales from training.")

    # Add missing IsHoliday column to test data if needed
    if 'IsHoliday' not in test_full.columns:
        print("   ⚠️ Adding missing IsHoliday column to test data with default value False")
        test_full['IsHoliday'] = False

    # Fill missing dates and sales for time series continuity
    print("   Filling missing dates and sales for time series continuity in training data...")

    unique_store_dept_dates = train_full[['Store', 'Dept', 'Date']].drop_duplicates()

    df_list = []
    for (store, dept), group in unique_store_dept_dates.groupby(['Store', 'Dept']):
        series_min_date = group['Date'].min()
        series_max_date = group['Date'].max()
        full_series_dates = pd.date_range(start=series_min_date, end=series_max_date, freq='W-FRI')

        temp_df = pd.DataFrame({
            'Store': store,
            'Dept': dept,
            'Date': full_series_dates
        })
        df_list.append(temp_df)

    complete_series_df = pd.concat(df_list, ignore_index=True)

    train_full_cleaned = pd.merge(
        complete_series_df,
        train_full,
        on=['Store', 'Dept', 'Date'],
        how='left'
    )

    nan_sales_before_fill = train_full_cleaned['Weekly_Sales'].isnull().sum()
    train_full_cleaned['Weekly_Sales'] = train_full_cleaned['Weekly_Sales'].fillna(0)
    print(f"   Filled {nan_sales_before_fill} NaN Weekly_Sales values with 0 for series continuity.")

    # Re-merge features and stores data
    train_full_cleaned = pd.merge(train_full_cleaned, features_df.drop(columns=['IsHoliday'], errors='ignore'), on=['Store', 'Date'], how='left', suffixes=('', '_feats'))
    train_full_cleaned = pd.merge(train_full_cleaned, stores_df, on=['Store'], how='left', suffixes=('', '_stores'))

    if 'IsHoliday_feats' in train_full_cleaned.columns:
        train_full_cleaned['IsHoliday'] = train_full_cleaned['IsHoliday'].fillna(train_full_cleaned['IsHoliday_feats'])
        train_full_cleaned = train_full_cleaned.drop(columns=['IsHoliday_feats'])

    # Fill NaNs in numerical features
    numerical_cols = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment'] + [f'MarkDown{i}' for i in range(1, 6)]

    for col in numerical_cols:
        if col in train_full_cleaned.columns:
            train_full_cleaned[col] = train_full_cleaned.groupby(['Store', 'Dept'])[col].transform(lambda x: x.fillna(x.mean()))
            train_full_cleaned[col] = train_full_cleaned[col].fillna(train_full_cleaned[col].mean())

        if col in test_full.columns:
            test_full[col] = test_full.groupby(['Store', 'Dept'])[col].transform(lambda x: x.fillna(x.mean()))
            test_full[col] = test_full[col].fillna(test_full[col].mean())

    # Final cleanup
    train_full_cleaned['Weekly_Sales'][train_full_cleaned['Weekly_Sales'] < 0] = 0
    train_full = train_full_cleaned.sort_values(by=['Date', 'Store', 'Dept']).reset_index(drop=True)
    test_full = test_full.sort_values(by=['Date', 'Store', 'Dept']).reset_index(drop=True)

    print(f"   ✅ Merged and cleaned training data: {train_full.shape}")
    print(f"   ✅ Merged and cleaned test data: {test_full.shape}")
    print(f"   📅 Train date range: {train_full['Date'].min()} to {train_full['Date'].max()}")
    print(f"   📅 Test date range: {test_full['Date'].min()} to {test_full['Date'].max()}")

    print("✅ STEP 1 COMPLETE!")
    return train_full, test_full

def step2_prepare_tft_data(train_full, test_full):
    """
    Step 2: Prepare data for TFT training and prediction
    Returns: X_train_full, y_train_full, X_test
    """
    print("\n📊 STEP 2: Preparing data for TFT training...")
    print("=" * 50)

    X_train_full = train_full.drop(columns=['Weekly_Sales']).copy()
    y_train_full = train_full['Weekly_Sales'].copy()
    X_test = test_full.copy()

    print(f"   📈 Training features: {X_train_full.shape}")
    print(f"   📈 Training targets: {y_train_full.shape}")
    print(f"   📉 Test features: {X_test.shape}")

    print("✅ STEP 2 COMPLETE!")
    return X_train_full, y_train_full, X_test

def step3_build_tft_pipeline():
    """
    Step 3: Build TFT preprocessing pipeline
    Returns: pipeline
    """
    print("\n⚙️ STEP 3: Building TFT preprocessing pipeline...")
    print("=" * 50)

    # Define column lists
    numerical_cols = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment'] + [f'MarkDown{i}' for i in range(1, 6)]
    categorical_ohe_cols = ["Type", "IsHoliday"]
    passthrough_cols = ["Store", "Dept"]

    numerical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean'))
    ])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    preprocessor = ColumnTransformerWithNames(transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_ohe_cols),
        ('pass', 'passthrough', passthrough_cols)
    ], remainder='drop')

    pipeline = Pipeline([
        ("multi_index_keeper", MultiIndexKeeper(index_cols=["Date", "Store", "Dept"])),
        ("date_feature_creator", DateFeatureCreator()),
        ("preprocessor", preprocessor),
        ("tft_regressor", TFTRegressor(
            input_chunk_length=52,
            output_chunk_length=39,
            epochs=25,
            batch_size=32,
            random_seed=42
        ))
    ])

    print("   ✅ Pipeline built successfully")
    print("✅ STEP 3 COMPLETE!")
    return pipeline

def step4_train_tft(pipeline, X_train_full, y_train_full):
    """
    Step 4: Train TFT model on full training dataset
    Returns: trained pipeline
    """
    print("\n🧠 STEP 4: Training TFT model on full dataset...")
    print("=" * 50)

    pipeline.fit(X_train_full, y_train_full)

    print("✅ STEP 4 COMPLETE!")
    print("   🎯 TFT model training finished on full dataset!")
    return pipeline

def step5_predict_tft(pipeline, X_test):
    """
    Step 5: Generate TFT predictions on test data
    Returns: test_predictions
    """
    print("\n🔮 STEP 5: Generating TFT predictions on test data...")
    print("=" * 50)

    test_predictions = pipeline.predict(X_test)
    test_predictions[test_predictions < 0] = 0

    print(f"   📊 Generated {len(test_predictions):,} predictions")
    print(f"   📊 Prediction statistics:")
    print(f"      Min: ${test_predictions.min():,.2f}")
    print(f"      Max: ${test_predictions.max():,.2f}")
    print(f"      Mean: ${test_predictions.mean():,.2f}")
    print(f"      Std: ${test_predictions.std():,.2f}")

    print("✅ STEP 5 COMPLETE!")
    return test_predictions

def step6_generate_submission(test_predictions, test_full, filename='submission_tft.csv'):
    """
    Step 6: Generate submission file in required format
    Returns: submission filename
    """
    print("\n📝 STEP 6: Generating submission file...")
    print("=" * 50)

    # Create submission DataFrame
    submission_df = test_full[['Store', 'Dept', 'Date']].copy()
    submission_df['Weekly_Sales'] = test_predictions

    # Create the Id column in the required format: Store_Dept_Date
    submission_df['Id'] = (submission_df['Store'].astype(str) + '_' +
                          submission_df['Dept'].astype(str) + '_' +
                          submission_df['Date'].dt.strftime('%Y-%m-%d'))

    # Create final submission in required format
    final_submission = submission_df[['Id', 'Weekly_Sales']].copy()

    # Save submission file
    final_submission.to_csv(filename, index=False)

    print(f"   ✅ Submission file saved: {filename}")
    print(f"   📊 Submission shape: {final_submission.shape}")
    print(f"   📝 Submission format preview:")
    print(final_submission.head())

    print("✅ STEP 6 COMPLETE!")
    print("🎉 ALL STEPS FINISHED!")
    print(f"📄 Download your submission file: {filename}")

    return filename

In [31]:
# Step 1: Load and prepare data
train_full, test_full = step1_load_and_prepare_data()

# Step 2: Prepare TFT data
X_train_full, y_train_full, X_test = step2_prepare_tft_data(train_full, test_full)


📊 STEP 1: Loading and preparing data...
   📂 Loading from local files...
   📈 Train data: (421570, 5)
   📉 Test data: (115064, 4)
   📊 Features data: (8190, 12)
   🏪 Stores data: (45, 3)
   🧹 Merging training data and initial cleaning...
   🧹 Merging test data...
   🗑️ Removed 1358 rows with negative Weekly_Sales from training.
   ⚠️ Adding missing IsHoliday column to test data with default value False
   Filling missing dates and sales for time series continuity in training data...
   Filled 26507 NaN Weekly_Sales values with 0 for series continuity.
   ✅ Merged and cleaned training data: (446719, 27)
   ✅ Merged and cleaned test data: (115064, 17)
   📅 Train date range: 2010-02-05 00:00:00 to 2012-10-26 00:00:00
   📅 Test date range: 2012-11-02 00:00:00 to 2013-07-26 00:00:00
✅ STEP 1 COMPLETE!

📊 STEP 2: Preparing data for TFT training...
   📈 Training features: (446719, 26)
   📈 Training targets: (446719,)
   📉 Test features: (115064, 17)
✅ STEP 2 COMPLETE!


In [32]:
# Step 3: Build TFT pipeline
pipeline = step3_build_tft_pipeline()



⚙️ STEP 3: Building TFT preprocessing pipeline...
   ✅ Pipeline built successfully
✅ STEP 3 COMPLETE!


In [33]:
pipeline = step4_train_tft(pipeline, X_train_full, y_train_full)




🧠 STEP 4: Training TFT model on full dataset...


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

✅ STEP 4 COMPLETE!
   🎯 TFT model training finished on full dataset!


In [34]:
test_predictions = step5_predict_tft(pipeline, X_test)



🔮 STEP 5: Generating TFT predictions on test data...
DEBUG: Found NaNs in futr_df_complete for columns: ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'Type_A', 'Type_B', 'Type_C', 'IsHoliday_False', 'IsHoliday_True', 'Store', 'Dept']. Filling with 0.


Predicting: |          | 0/? [00:00<?, ?it/s]

DEBUG: Found NaNs in final y_pred after reindex. Filling with 0.
   📊 Generated 115,064 predictions
   📊 Prediction statistics:
      Min: $0.00
      Max: $264,611.59
      Mean: $16,009.67
      Std: $22,182.98
✅ STEP 5 COMPLETE!


In [35]:
filename = step6_generate_submission(test_predictions, test_full, 'submission_tft.csv')


📝 STEP 6: Generating submission file...
   ✅ Submission file saved: submission_tft.csv
   📊 Submission shape: (115064, 2)
   📝 Submission format preview:
               Id  Weekly_Sales
0  1_1_2012-11-02  21702.884766
1  1_2_2012-11-02  44117.097656
2  1_3_2012-11-02   9480.584961
3  1_4_2012-11-02  37046.226562
4  1_5_2012-11-02  25508.361328
✅ STEP 6 COMPLETE!
🎉 ALL STEPS FINISHED!
📄 Download your submission file: submission_tft.csv
